# Library

In [34]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import math
import xlsxwriter

In [35]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# fold path

In [36]:
drive = 'D'
raw_fold = f'{drive}:/data/big2/BD1/raw'
ods_fold = f'{drive}:/data/big2/ODS'
sample_fold = f'{drive}:/data/big2/BD1/sample'
analysis_fold = f'{drive}:/data/big2/BD1/an'

# Load

## 병합 result 최종 파일

In [37]:
# about 18.9s
name = '4등급_차량_리스트_양식_result(전체)_Grade_추가(2023.04.18_PM.03.07.04)'
file_name = f'{name}.csv'
df = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False, encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 37 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   차대번호        1502235 non-null  object 
 1   차량번호        1502235 non-null  object 
 2   제원관리번호      1502235 non-null  object 
 3   차종          1502235 non-null  object 
 4   용도          1502235 non-null  object 
 5   최초등록일       1502235 non-null  int64  
 6   연식          1502235 non-null  int64  
 7   제작일자        1335178 non-null  float64
 8   검사유효일       1502223 non-null  float64
 9   배출가스인증번호    1384783 non-null  object 
 10  등급          1502235 non-null  int64  
 11  차명          1502235 non-null  object 
 12  차종분류        1502235 non-null  object 
 13  차종유형        1502235 non-null  object 
 14  자동차형식       1501958 non-null  object 
 15  제작사명        1492590 non-null  object 
 16  연료          1502235 non-null  object 
 17  엔진형식        1500400 non-null  object 
 18  총중량         1502234 no

# 전처리

In [38]:
df.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade'],
      dtype='object')

In [39]:
df['연료'].value_counts(dropna=False)

경유             1153813
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [40]:
# 한불모터스에서 확인 한 1대 3등급으로 변경
# 3등급 : 4396 + 1 = 4397
df['등급_수정'].value_counts(dropna=False)

4.00    1497711
3.00       4397
5.00        127
Name: 등급_수정, dtype: int64

## 4등급 차량(휘발유, 경유, LPG) 추출

In [41]:
df1 = df[df['등급_수정'] == 4].reset_index(drop=True)
df1.shape

(1497711, 37)

In [42]:
g3_g5 = df[df['등급_수정'] != 4].reset_index(drop=True)
g3_g5.shape

(4524, 37)

In [43]:
df1['연료'].value_counts(dropna=False)

경유             1149289
휘발유             323019
LPG(액화석유가스)      25400
알코올                  1
기타연료                 1
CNG(압축천연가스)          1
Name: 연료, dtype: int64

In [44]:
df1[(df1['연료'] == '휘발유') | (df1['연료'] == '경유') | (df1['연료'] == 'LPG(액화석유가스)')].shape

(1497708, 37)

In [45]:
df1 = df1[(df1['연료'] == '휘발유') | (df1['연료'] == '경유') | (df1['연료'] == 'LPG(액화석유가스)')].reset_index(drop=True)

In [46]:
df1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade'],
      dtype='object')

In [47]:
df1['연료'].value_counts(dropna=False)

경유             1149289
휘발유             323019
LPG(액화석유가스)      25400
Name: 연료, dtype: int64

## 중복 차대번호 확인
- 최종등록일자, 배출가스검사유효일자 최신 데이터 사용

In [48]:
df1.shape[0], len(df1['차대번호'].unique())

(1497708, 1497687)

## 차종 확인

In [49]:
df1['차종'].value_counts(dropna=False)

승용    852363
화물    496078
승합    142815
특수      6452
Name: 차종, dtype: int64

## 차종유형 확인

In [50]:
df1['차종유형'].value_counts()

중형    705734
소형    554911
대형    209816
경형     27247
Name: 차종유형, dtype: int64

## 연식 확인

In [51]:
df1['연식'].describe()

count   1497708.00
mean       2005.87
std           6.83
min        1901.00
25%        2006.00
50%        2009.00
75%        2010.00
max        2018.00
Name: 연식, dtype: float64

## 제작일자 확인

In [52]:
df1['제작일자'].describe()

count    1330654.00
mean    20072621.19
std        52791.07
min         1993.00
25%     20070917.00
50%     20090406.00
75%     20100616.00
max     20190923.00
Name: 제작일자, dtype: float64

## 연료별 차종별 차종유형 확인

In [53]:
df1.groupby(['연료', '차종', '차종유형'])['차대번호'].count()

연료           차종  차종유형
LPG(액화석유가스)  승용  경형          51
                 대형         845
                 소형         131
                 중형       11221
             승합  경형        2148
                 소형         386
                 중형        4501
             특수  중형           2
             화물  경형        3511
                 대형           1
                 소형        1792
                 중형         811
경유           승용  경형           1
                 대형      146277
                 소형       14589
                 중형      364145
             승합  대형        3470
                 중형      129054
             특수  대형        3877
                 소형        2045
                 중형         524
             화물  대형       20782
                 소형      426616
                 중형       37909
휘발유          승용  경형       17519
                 대형       34562
                 소형      106300
                 중형      156722
             승합  경형        2010
                 대형           1
                 소

In [54]:
for fuel in ['경유', '휘발유', 'LPG(액화석유가스)']:
    temp1 = df1[df1['연료'] == fuel].reset_index(drop=True)
    for car_type in temp1['차종'].unique():
        temp2 = temp1[temp1['차종'] == car_type].reset_index(drop=True)
        car_size = list(temp2['차종유형'].unique())
        print(f'{fuel} - {car_type}의 차종유형 : {car_size}')

경유 - 승용의 차종유형 : ['대형', '중형', '소형', '경형']
경유 - 특수의 차종유형 : ['대형', '소형', '중형']
경유 - 승합의 차종유형 : ['대형', '중형']
경유 - 화물의 차종유형 : ['대형', '소형', '중형']
휘발유 - 승용의 차종유형 : ['소형', '대형', '중형', '경형']
휘발유 - 화물의 차종유형 : ['경형', '소형', '대형', '중형']
휘발유 - 승합의 차종유형 : ['경형', '소형', '중형', '대형']
휘발유 - 특수의 차종유형 : ['중형']
LPG(액화석유가스) - 승합의 차종유형 : ['중형', '경형', '소형']
LPG(액화석유가스) - 승용의 차종유형 : ['중형', '대형', '소형', '경형']
LPG(액화석유가스) - 화물의 차종유형 : ['소형', '경형', '중형', '대형']
LPG(액화석유가스) - 특수의 차종유형 : ['중형']


### 차령 정보 계산 및 데이터 추가

In [55]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [56]:
df1['차령'] = current_yr - df1['연식']
df1['차령'].describe()

count   1497708.00
mean         17.13
std           6.83
min           5.00
25%          13.00
50%          14.00
75%          17.00
max         122.00
Name: 차령, dtype: float64

## 시도명, 시군구명 수정
- 시군구명 앞지역명만 추출
- 세종특별자치시 -> 세종시
- 여주군 -> 여주시
- 청원군 -> 청원시
- 인천광역시 남구 -> 미추홀구

In [57]:
df1[['시도명', '시군구명', '차종']].isnull().sum()

시도명     0
시군구명    0
차종      0
dtype: int64

In [58]:
df1['시도명'].unique()

array(['대전광역시', '부산광역시', '경기도', '제주특별자치도', '경상남도', '강원도', '전라남도', '인천광역시',
       '서울특별시', '충청북도', '충청남도', '대구광역시', '전라북도', '울산광역시', '경상북도', '광주광역시',
       '세종특별자치시'], dtype=object)

In [59]:
df1[['시도명', '시군구명']].iloc[10:20]

,시도명,시군구명
10,경기도,남양주시
11,경기도,화성시
12,경기도,안성시
13,경기도,평택시
14,경기도,안산시 단원구
15,부산광역시,부산진구
16,인천광역시,강화군
17,서울특별시,서초구
18,부산광역시,금정구
19,경기도,성남시 분당구


In [60]:
df1['시군구명_수정'] = df1['시군구명'].str.split(' ').str[0]

In [61]:
df1[['시도명', '시군구명', '시군구명_수정']].iloc[10:20]

,시도명,시군구명,시군구명_수정
10,경기도,남양주시,남양주시
11,경기도,화성시,화성시
12,경기도,안성시,안성시
13,경기도,평택시,평택시
14,경기도,안산시 단원구,안산시
15,부산광역시,부산진구,부산진구
16,인천광역시,강화군,강화군
17,서울특별시,서초구,서초구
18,부산광역시,금정구,금정구
19,경기도,성남시 분당구,성남시


In [62]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(8102,)

In [63]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'] = '세종시'

In [64]:
df1.loc[df1['시군구명_수정'] == '세종특별자치시', '시군구명_수정'].shape

(0,)

In [65]:
df1.loc[df1['시군구명_수정'] == '여주군', '시군구명_수정'].shape

(1,)

In [66]:
df1.loc[df1['시군구명_수정'] == '여주군', '시군구명_수정'] = '여주시'

In [67]:
df1.loc[df1['시군구명_수정'] == '여주군', '시군구명_수정'].shape

(0,)

In [68]:
df1.loc[df1['시군구명_수정'] == '청원군', '시군구명_수정'].shape

(4,)

In [69]:
df1.loc[df1['시군구명_수정'] == '청원군', '시군구명_수정'] = '청주시'

In [70]:
df1.loc[df1['시군구명_수정'] == '청원군', '시군구명_수정'].shape

(0,)

In [71]:
df1.loc[(df1['시도명'] == '인천광역시') & (df1['시군구명_수정'] == '남구'), '시군구명_수정'].shape

(9,)

In [72]:
df1.loc[(df1['시도명'] == '인천광역시') & (df1['시군구명_수정'] == '남구'), '시군구명_수정'] = '미추홀구'

In [73]:
df1.loc[(df1['시도명'] == '인천광역시') & (df1['시군구명_수정'] == '남구'), '시군구명_수정'].shape

(0,)

# 자동차-엔진 가열(Hot-start) 배출

## 배출량 식
$$E_{i, j} = VKT \times {EF_i \over {1000}} \times DF \times (1 - {R \over {100}})$$

$E_{i, j}$ : 자동차 $j$의 도로주행시 발생한 오염물질 $i$의 배출량($kg/yr$)

$VKT$ : 주행거리($km/yr$)

$EF_i$ : 차종별, 연료별, 연식별, 차속별 배출계수($g/km$)

$DF$ : 열화계수

$R$ : 저감장치 부착 효율(%)

## 주행거리(VKT)
- [현재 설정] "용도별_차종별_시군구별_자동차주행거리" 자료(KOSIS)(2021)의 1일 평균주행거리 * 365 = 주행거리(km/yr)

### [로드] 1일평균주행거리(2021)

In [74]:
# 1일 평균 주행거리 자료 로드
run = pd.read_excel(os.path.join(raw_fold, '용도별_차종별_시군구별_자동차1일평균주행거리(2021).xlsx'))
run.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1230 entries, 0 to 1229
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시도명     1230 non-null   object 
 1   시군구명    1230 non-null   object 
 2   차종      1230 non-null   object 
 3   전체      1230 non-null   float64
dtypes: float64(1), object(3)
memory usage: 38.6+ KB


In [75]:
run.tail()

,시도명,시군구명,차종,전체
1225,제주특별자치도,서귀포시,합계,35.10
1226,제주특별자치도,서귀포시,승용,36.10
1227,제주특별자치도,서귀포시,승합,37.20
1228,제주특별자치도,서귀포시,화물,32.50
1229,제주특별자치도,서귀포시,특수,38.40


In [76]:
len(set(df1['시군구명_수정'].unique()) - set(run['시군구명'].unique()))

0

In [77]:
df1.shape

(1497708, 39)

### [병합] 1일평균주행거리 데이터 추가(df1)

In [78]:
df2 = df1.merge(run, left_on=['시도명', '시군구명_수정', '차종'], right_on=['시도명', '시군구명', '차종'], how='left')
df2.shape

(1497708, 41)

In [79]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명_x', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '시군구명_y', '전체'],
      dtype='object')

In [80]:
df2.drop('시군구명_y', axis=1, inplace=True)
df2.rename(columns={'시군구명_x':'시군구명', '전체':'1일평균주행거리'}, inplace=True)
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '1일평균주행거리'],
      dtype='object')

In [81]:
df2['1일평균주행거리'].isnull().sum()

0

In [82]:
df2[['시도명', '시군구명', '시군구명_수정', '차종', '1일평균주행거리']].tail()

,시도명,시군구명,시군구명_수정,차종,1일평균주행거리
1497703,대구광역시,달성군,달성군,승용,37.90
1497704,대구광역시,북구,북구,승용,36.80
1497705,대구광역시,동구,동구,승용,37.40
1497706,대구광역시,서구,서구,승용,36.50
1497707,대구광역시,달서구,달서구,승용,36.20


#### [출력] 1일평균주행거리 추가 데이터

In [83]:
# today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
# today_date

In [84]:
# # about 24.1s
# name = f'4등급_차량_리스트_양식_result(전체)_Grade_추가_1일평균주행거리_추가({today_date})'
# file_name = f'{name}.csv'
# df1.to_csv(os.path.join(raw_fold, file_name), index=False)

# 로드

In [85]:
# # about 14.1s
# name = f'4등급_차량_리스트_양식_result(전체)_Grade_추가_1일평균주행거리_추가(2023.04.19_PM.02.25.59)'
# file_name = f'{name}.csv'
# df1 = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)

In [86]:
df2['VKT'] = df2['1일평균주행거리'] * 365

In [87]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '1일평균주행거리', 'VKT'],
      dtype='object')

In [88]:
df2[['시도명', '시군구명', '차종', 'VKT']].tail()

,시도명,시군구명,차종,VKT
1497703,대구광역시,달성군,승용,13833.50
1497704,대구광역시,북구,승용,13432.00
1497705,대구광역시,동구,승용,13651.00
1497706,대구광역시,서구,승용,13322.50
1497707,대구광역시,달서구,승용,13213.00


In [89]:
df2['VKT'].isnull().sum()

0

## DF(열화계수)
- 필요한 정보 : 연료, 차종, 차종유형, 연식, 차령
- 조건 수정(2023.04.20, 최)
    - 특수 소형 = 승용 소형
    - 승합 대형 = 화물 대형

In [90]:
df2['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [91]:
DF_col = ['연료', '차종', '차종유형', '연식', '차령']

In [92]:
df2[DF_col].head()

,연료,차종,차종유형,연식,차령
0,경유,승용,대형,2006,17
1,경유,승용,대형,2006,17
2,경유,승용,대형,2006,17
3,경유,승용,대형,2006,17
4,경유,승용,대형,2006,17


In [93]:
df2[DF_col].values[0]

array(['경유', '승용', '대형', 2006, 17], dtype=object)

### ❗ DF(열화계수) 코드
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 열화계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1

In [94]:
# about 5.0s
DF_CO_list = []
DF_HC_list = []
DF_PM_list = []
DF_NOx_list = []
# '연료', '차종', '차종유형', '연식', '차령'
for fuel, car_type, car_size, car_birth, car_age in tqdm(df2[DF_col].values):    
    if fuel == '경유':
        if (car_type == '승용') or ( (car_type == '특수') and (car_size == '소형') ): # 최 확인(2023.04.20) : "특수 소형" -> "승용 소형" 조건으로 계산
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '승합':
            if car_size == '소형' or car_size == '중형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
            elif car_size == '대형':
                warranty = 3
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.05
                    DF_HC = 1 + (car_age - warranty)*0.05
                    DF_PM = 1 + (car_age - warranty)*0.05
                    DF_NOx = 1 + (car_age - warranty)*0.02
                    if DF_CO > 1.5:
                        DF_CO = 1.5
                    if DF_HC > 1.5:
                        DF_HC = 1.5
                    if DF_PM > 1.5:
                        DF_PM = 1.5
                    if DF_NOx > 1.2:
                        DF_NOx = 1.2
        elif car_type == '화물':
            # car_size : 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
        elif car_type == '특수':
            # car_size : 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.05
                DF_HC = 1 + (car_age - warranty)*0.05
                DF_PM = 1 + (car_age - warranty)*0.05
                DF_NOx = 1 + (car_age - warranty)*0.02
                if DF_CO > 1.5:
                    DF_CO = 1.5
                if DF_HC > 1.5:
                    DF_HC = 1.5
                if DF_PM > 1.5:
                    DF_PM = 1.5
                if DF_NOx > 1.2:
                    DF_NOx = 1.2
    elif (fuel == '휘발유') or (fuel == 'LPG(액화석유가스)'):
        if car_type == '승용':
            # 경, 소, 중, 대
            if car_birth <= 2000:
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif car_birth >= 2001:
                warranty = 10
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
        elif car_type == '승합':
            if car_size == '경형':
                warranty = 5
                if car_age <= warranty:
                    DF_CO = 1
                    DF_HC = 1
                    DF_PM = 1
                    DF_NOx = 1
                else:
                    DF_CO = 1 + (car_age - warranty)*0.1
                    DF_HC = 1 + (car_age - warranty)*0.1
                    DF_PM = 1 + (car_age - warranty)*0.1
                    DF_NOx = 1 + (car_age - warranty)*0.1
                    if DF_CO > 2.0:
                        DF_CO = 2.0
                    if DF_HC > 2.0:
                        DF_HC = 2.0
                    if DF_PM > 2.0:
                        DF_PM = 2.0
                    if DF_NOx > 2.0:
                        DF_NOx = 2.0
            elif (car_size == '소형') or (car_size == '중형'):
                if car_birth <= 2005:
                    warranty = 5
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
                elif car_birth >= 2006:
                    warranty = 10
                    if car_age <= warranty:
                        DF_CO = 1
                        DF_HC = 1
                        DF_PM = 1
                        DF_NOx = 1
                    else:
                        DF_CO = 1 + (car_age - warranty)*0.1
                        DF_HC = 1 + (car_age - warranty)*0.1
                        DF_PM = 1 + (car_age - warranty)*0.1
                        DF_NOx = 1 + (car_age - warranty)*0.1
                        if DF_CO > 2.0:
                            DF_CO = 2.0
                        if DF_HC > 2.0:
                            DF_HC = 2.0
                        if DF_PM > 2.0:
                            DF_PM = 2.0
                        if DF_NOx > 2.0:
                            DF_NOx = 2.0
        elif (car_type == '화물') or ( (car_type == '승합') and (car_size == '대형')): # 최 확인(2023.04.20) : 습합 대형 -> 화물 대형 조건으로 계산
            # 경, 소, 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
        elif car_type == '특수':
            # 중, 대
            warranty = 5
            if car_age <= warranty:
                DF_CO = 1
                DF_HC = 1
                DF_PM = 1
                DF_NOx = 1
            else:
                DF_CO = 1 + (car_age - warranty)*0.1
                DF_HC = 1 + (car_age - warranty)*0.1
                DF_PM = 1 + (car_age - warranty)*0.1
                DF_NOx = 1 + (car_age - warranty)*0.1
                if DF_CO > 2.0:
                    DF_CO = 2.0
                if DF_HC > 2.0:
                    DF_HC = 2.0
                if DF_PM > 2.0:
                    DF_PM = 2.0
                if DF_NOx > 2.0:
                    DF_NOx = 2.0
    else:
        # 설정 고민
            # 고려 : 1
        # DF_CO, DF_HC, DF_PM, DF_NOx = np.nan, np.nan, np.nan, np.nan
        DF_CO, DF_HC, DF_PM, DF_NOx = 1, 1, 1, 1

    # 열화계수(DF) 리스트에 저장
    DF_CO_list.append(DF_CO)
    DF_HC_list.append(DF_HC)
    DF_PM_list.append(DF_PM)
    DF_NOx_list.append(DF_NOx)

100%|██████████| 1497708/1497708 [00:04<00:00, 316478.00it/s]


In [95]:
df2.shape[0]

1497708

In [96]:
len(DF_CO_list), len(DF_HC_list), len(DF_PM_list), len(DF_NOx_list)

(1497708, 1497708, 1497708, 1497708)

In [97]:
df2['DF_CO'] = DF_CO_list
df2['DF_HC'] = DF_HC_list
df2['DF_PM'] = DF_PM_list
df2['DF_NOx'] = DF_NOx_list

In [98]:
check_DF_col = ['DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx']
df2.loc[df2['차령'] < 10, DF_col + check_DF_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx
1140557,경유,화물,중형,2018,5,1.00,1.00,1.00,1.00
1143212,경유,화물,대형,2018,5,1.00,1.00,1.00,1.00


## 저감장치 부착 효율(R) 계산
- DPF유무_수정 : 유 -> DPF로 가정하고 진행
- 휘발성 유기 화합물(Volatile Organic Compounds:VOC) : 생활주변에서 흔히 사용하는 탄화수소류가 거의 해당됨.
- 저감장치별 물질 제거 효율(%)

    장치종류|CO|VOC|PM
    :-:|:-:|:-:|:-:
    DPF|99.5|90|83.6
    pDPF|94.6|89.3|56
    DOC|85.4|72|35

### 현대자동차 상용차 중 pDPF 달린 차량
- 6MY-HD-23-55
- 7MY-HD-24-28
- 7MY-HD-24-30

In [99]:
df2['DPF유무_수정'].value_counts(dropna=False)

무       879365
NaN     348419
유       253675
확인불가     16249
Name: DPF유무_수정, dtype: int64

In [100]:
df2['DPF유무_수정'].values[:5]

array(['무', '무', '무', '무', '무'], dtype=object)

In [101]:
df2['DPF유무_수정'].values[0]

'무'

In [102]:
df2.loc[(df2['배인번호_수정'] == '6MY-HD-23-55') & (df2['제작사명'] == '현대자동차(주)')].shape

(0, 45)

In [103]:
df2.loc[(df2['배인번호_수정'] == '7MY-HD-24-28') & (df2['제작사명'] == '현대자동차(주)')].shape

(24413, 45)

In [104]:
df2.loc[(df2['배인번호_수정'] == '7MY-HD-24-30') & (df2['제작사명'] == '현대자동차(주)')].shape

(10751, 45)

In [105]:
df2.loc[df2['DPF유무_수정'] == '유', ['R_CO', 'R_HC', 'R_PM']] = 99.5, 90, 83.6
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '6MY-HD-23-55') & (df2['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '7MY-HD-24-28') & (df2['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '7MY-HD-24-30') & (df2['제작사명'] == '현대자동차(주)'), ['R_CO', 'R_HC', 'R_PM']] = 94.6, 89.3, 56

In [106]:
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '6MY-HD-23-55') & (df2['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

Series([], Name: R_PM, dtype: float64)

In [107]:
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '7MY-HD-24-28') & (df2['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

734704   56.00
734705   56.00
734706   56.00
734707   56.00
734708   56.00
Name: R_PM, dtype: float64

In [108]:
df2.loc[(df2['DPF유무_수정'] == '유') & (df2['배인번호_수정'] == '7MY-HD-24-30') & (df2['제작사명'] == '현대자동차(주)'), 'R_PM'].head()

766085   56.00
766086   56.00
766087   56.00
766088   56.00
766089   56.00
Name: R_PM, dtype: float64

In [109]:
df2.shape[0]

1497708

In [110]:
check_R_col = ['R_CO', 'R_HC', 'R_PM']

In [111]:
df2[check_R_col].notnull().sum()

R_CO    253675
R_HC    253675
R_PM    253675
dtype: int64

In [112]:
df2.loc[df2['DPF유무_수정'] == '유', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
3207,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90.00,83.60
3208,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90.00,83.60
3209,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
3210,경유,승용,대형,2007,16,1.50,1.50,1.50,1.20,99.50,90.00,83.60
3211,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,99.50,90.00,83.60


In [113]:
df2.loc[df2['DPF유무_수정'] == '무', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
0,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,NaN,NaN,NaN
1,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,NaN,NaN,NaN
2,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,NaN,NaN,NaN
3,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,NaN,NaN,NaN
4,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,NaN,NaN,NaN


In [114]:
df2.loc[df2['DPF유무_수정'] == '확인불가', DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
43680,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,NaN,NaN,NaN
43687,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,NaN,NaN,NaN
43690,경유,승용,중형,2008,15,1.50,1.50,1.50,1.20,NaN,NaN,NaN
71238,경유,승용,중형,2009,14,1.45,1.45,1.45,1.18,NaN,NaN,NaN
71239,경유,승용,중형,2007,16,1.50,1.50,1.50,1.20,NaN,NaN,NaN


In [115]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [116]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

### DPF 무, 확인불가, nan의 R값 수정
- 0

In [117]:
df2[check_R_col] = df2[check_R_col].fillna(0)

In [118]:
df2[check_R_col].isnull().sum()

R_CO    0
R_HC    0
R_PM    0
dtype: int64

In [119]:
df2.loc[df2['DPF유무_수정'].isnull() == True, DF_col + check_DF_col + check_R_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM
1149289,휘발유,승용,소형,2001,22,2.00,2.00,2.00,2.00,0.00,0.00,0.00
1149290,휘발유,승용,대형,1999,24,2.00,2.00,2.00,2.00,0.00,0.00,0.00
1149291,휘발유,승용,소형,1991,32,2.00,2.00,2.00,2.00,0.00,0.00,0.00
1149292,LPG(액화석유가스),승합,중형,2000,23,2.00,2.00,2.00,2.00,0.00,0.00,0.00
1149293,휘발유,승용,중형,1999,24,2.00,2.00,2.00,2.00,0.00,0.00,0.00


## ❗ 배출계수(EFi)
- 연료, 차종, 차종유형 설정에 해당되지 않는 차량의 배출계수 값 설정
    - 현재 설정값 : np.nan
    - 고려 설정값 : 1
- 조건 수정(2023.04.20, 최)
    - 특수 중형 = 승합 중형
    - 화물 중형 = 승합 중형
    - 화물 대형 = 승합 대형
    - 특수 중형 = 승용 중형

In [120]:
EFi_col = ['차종', '차종유형', '연료', '연식']
# 참고 : KOSIS 차량속도(2017) 일반국도 평균 (https://kosis.kr/statHtml/statHtml.do?orgId=210&tblId=DT_21002_J008)
V = 54.1
# 국가 대기오염물질 배출량 산정방법 편람(V)(2022) 부록 참고(for PM-2.5)
k = 0.92

In [121]:
df2[EFi_col].values[0]

array(['승용', '대형', '경유', 2006], dtype=object)

In [122]:
df2['차종유형'].unique()

array(['대형', '중형', '소형', '경형'], dtype=object)

In [123]:
# about 7.9s
EFi_CO_list = []
EFi_HC_list = []
EFi_NOx_list = []
EFi_PM10_list = []
EFi_PM2_5_list = []

for car_type, car_size, fuel, car_birth in tqdm(df2[EFi_col].values):
    if fuel == '휘발유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 65:
                    EFi_CO = 59.783 * (V**-1.0007)
                else:
                    EFi_CO = 0.0874 * V - 3.5618
                EFi_HC = 8.6275 * (V**-1.0722)
                EFi_NOx = 3.2294 * (V**-0.5763)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 5.1835 * (V**-1.1889)
                EFi_NOx = 1.7525 * (V**-0.6481)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 65:
                    EFi_CO = 60.556 * (V**-1.2501)
                else:
                    EFi_CO = -0.0006 * V + 0.5753
                EFi_HC = 0.7446 * (V**-0.9392)
                EFi_NOx = 0.3403 * (V**-0.5455)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 45:
                    EFi_CO = 4.9952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * V**2 + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 45:
                    EFi_CO = 4.5956 * (V**-0.8461)
                else:
                    EFi_CO = -9.2000*(10**-5) * (V**2) + 2.1068*(10**-2) * V - 5.2449*(10**-1)
                EFi_HC = 0.2662 * (V**-0.7830)
                EFi_NOx = 0.4476 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 45:
                    EFi_CO = 4.4517 * (V**-0.8461)
                else:
                    EFi_CO = -8.9120*(10**-5) * (V**2) + 2.0408*(10**-2)*V - 5.0807*(10**-1)
                EFi_HC = 0.2556 * (V**-0.7830)
                EFi_NOx = 0.4353 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 45:
                    EFi_CO = 4.3079 * (V**-0.8461)
                else:
                    EFi_CO = -8.6240*(10**-5) * (V**2) + 1.9749*(10**-2)*V - 4.9165*(10**-1)
                EFi_HC = 0.2449 * (V**-0.7830)
                EFi_NOx = 0.4230 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.164 * (V**-0.8461)
                else:
                    EFi_CO = -8.3360*(10**-5) * (V**2) + 1.9089*(10**-2)*V - 4.7524*(10**-1)
                EFi_HC = 0.2343 * (V**-0.7830)
                EFi_NOx = 0.4106 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 111.67 * (V**-1.1566)
                EFi_HC = 32.017 * (V**-1.4171)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 22.356 * (V**-0.9068)
                EFi_HC = 0.8428 * (V**-0.8829)
                EFi_NOx = 1.2613 * (V**-0.3873)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 1.4898 * (V**-0.3837)
                EFi_HC = 0.1738 * (V**-0.7268)
                EFi_NOx = 0.1563 * (V**-0.2671)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 1.0000*(10**-4)*(V**2) - 7.1000*(10**-3)*V + 2.2450*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 9.2000*(10**-5)*(V**2) - 6.5320*(10**-3)*V + 2.0654*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 8.9120*(10**-5)*(V**2) - 6.3275*(10**-3)*V + 2.0007*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 8.6240*(10**-5)*(V**2) - 6.1230*(10**-3)*V + 1.9361*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 8.3360*(10**-5)*(V**2) - 5.9186*(10**-3)*V + 1.8714*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.09840*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승용') and (car_size == '중형') ) or ( (car_type == '특수') and (car_size == '중형') ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                EFi_CO = 2.2900*(10**-5)*(V**2) - 1.6300*(10**-3)*V + 5.8300*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                EFi_CO = 2.1068*(10**-5)*(V**2) - 1.4996*(10**-3)*V + 5.3636*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                EFi_CO = 2.0408*(10**-5)*(V**2) - 1.4527*(10**-3)*V + 5.1957*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                EFi_CO = 1.9749*(10**-5)*(V**2) - 1.4057*(10**-3)*V + 5.0278*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                EFi_CO = 1.9089*(10**-5)*(V**2) - 1.3588*(10**-3)*V + 4.8599*(10**-2)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 1986:
                EFi_CO = 247.00 * (V**-0.6651)
                EFi_HC = 15.953 * (V**-0.5059)
                EFi_NOx = 3.1140 * (V**-0.2278)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1987 <= car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1999:
                EFi_CO = 51.555 * (V**-0.9531)
                EFi_HC = 31.816 * (V**-1.4804)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 29.921 * (V**-0.8868)
                EFi_HC = 7.9374 * (V**-1.3041)
                EFi_NOx = 1.8525 * (V**-0.4192)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 2.4938 * (V**-0.6106)
                EFi_HC = 0.4262 * (V**-1.0122)
                EFi_NOx = 0.1818 * (V**-0.4316)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth <= 2008:
                if V <= 65.4:
                    EFi_CO = 1.4082 * (V**-0.7728)
                else:
                    EFi_CO = 8.0000*(10**-5)*(V**2) - 1.2700*(10**-2)*V + 5.7510*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0633 * (V**-1.0484)
                else:
                    EFi_HC = 1.3200*(10**-6)*(V**2) - 1.8800*(10**-4)*V + 7.7000*(10**-3)
                EFi_NOx = -3.5000*(10**-6)*(V**2) + 3.3000*(10**-4)*V + 1.1200*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2009 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 1.2955 * (V**-0.7728)
                else:
                    EFi_CO = 7.3600*(10**-5)*(V**2) - 1.1684*(10**-2)*V + 5.2909*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0570 * (V**-1.0484)
                else:
                    EFi_HC = 1.1880*(10**-6)*(V**2) - 1.6920*(10**-4)*V + 6.9300*(10**-3)
                EFi_NOx = -3.2511*(10**-6)*(V**2) + 3.0653*(10**-4)*V + 1.0404*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2012 <= car_birth <= 2013:
                if V <= 65.4:
                    EFi_CO = 1.2550 * (V**-0.7728)
                else:
                    EFi_CO = 7.1296*(10**-5)*(V**2) - 1.1318*(10**-2)*V + 5.1253*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0547 * (V**-1.0484)
                else:
                    EFi_HC = 1.1405*(10**-6)*(V**2) - 1.6243*(10**-4)*V + 6.6528*(10**-3)
                EFi_NOx = -3.1615*(10**-6)*(V**2) + 2.9809*(10**-4)*V + 1.0117*(10**-2)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2014:
                if V <= 65.4:
                    EFi_CO = 1.2144 * (V**-0.7728)
                else:
                    EFi_CO = 6.8992*(10**-5)*(V**2) - 1.0952*(10**-2)*V + 4.9597*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0524 * (V**-1.0484)
                else:
                    EFi_HC = 1.0930*(10**-6)*(V**2) - 1.5566*(10**-4)*V + 6.3756*(10**-3)
                EFi_NOx = -3.0719*(10**-6)*(V**2) + 2.8964*(10**-4)*V + 9.8301*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2015 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 1.1739 * (V**-0.7728)
                else:
                    EFi_CO = 6.6688*(10**-5)*(V**2) - 1.0587*(10**-2)*V + 4.7940*(10**-1)
                if V <= 65.4:
                    EFi_HC = 0.0501 * (V**-1.0484)
                else:
                    EFi_HC = 1.0500*(10**-6)*(V**2) - 1.4890*(10**-4)*V + 6.0984*(10**-3)
                EFi_NOx = -2.9823*(10**-6)*(V**2) + 2.8119*(10**-4)*V + 9.5434*(10**-3)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 11.249 * (V**-0.6579)
                else:
                    EFi_CO = 0.0003 * (V**2) + 0.0002 * V + 0.4136
                EFi_HC = 7.6244 * (V**-0.8364)
                EFi_NOx = 2.6754 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 16.965 * (V**-0.8461)
                else:
                    EFi_CO = -0.0003 * (V**2) + 0.0777 * V - 1.9363
                EFi_HC = 3.0285 * (V**-0.7830)
                EFi_NOx = 1.9923 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 1.8928 * (V**-0.7830)
                EFi_NOx = 1.2352 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 9.9433 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0455 * V - 1.1349
                EFi_HC = 0.9227 * (V**-0.7830)
                EFi_NOx = 3.8859 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 4.4952 * (V**-0.8461)
                else:
                    EFi_CO = -0.0001 * (V**2) + 0.0229 * V - 0.5701
                EFi_HC = 0.2958 * (V**-0.7830)
                EFi_NOx = 0.4819 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif car_birth == 2000:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2001 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 11.920 * (V**-1.5356)
                EFi_NOx = 3.0649 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 37.573 * (V**-1.2078)
                EFi_HC = 3.1786 * (V**-1.5356)
                EFi_NOx = 1.4931 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 32.899 * (V**-1.2078)
                EFi_HC = 2.7387 * (V**-1.5356)
                EFi_NOx = 1.1808 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif ( (car_type == '승합') and ((car_size == '중형') or (car_size == '대형')) ) or ( (car_type == '화물') and (car_size == '소형') ):
            if car_birth <= 1990:
                EFi_CO = 36.169 * (V**-0.7587)
                EFi_HC = 15.607 * (V**-1.0423)
                EFi_NOx = 6.2007 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1991 <= car_birth <= 1996:
                EFi_CO = 39.402 * (V**-0.8879)
                EFi_HC = 23.400 * (V**-1.4041)
                EFi_NOx = 7.5244 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 77.088 * (V**-1.2078)
                EFi_HC = 18.731 * (V**-1.5356)
                EFi_NOx = 4.4260 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 41.669 * (V**-1.2078)
                EFi_HC = 14.190 * (V**-1.5356)
                EFi_NOx = 3.4578 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 36.578 * (V**-1.2078)
                EFi_HC = 3.0337 * (V**-1.5356)
                EFi_NOx = 2.0104 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
            elif 2006 <= car_birth:
                EFi_CO = 14.202 * (V**-1.2078)
                EFi_HC = 1.2233 * (V**-1.5356)
                EFi_NOx = 0.2493 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
        elif (car_type == '화물') and ( (car_size == '중형') or (car_size == '대형') ):
            if 1993 <= car_birth:
                if V <= 65:
                    EFi_CO = 70
                    EFi_HC = 7
                    EFi_NOx = 4.5
                else:
                    EFi_CO = 55
                    EFi_HC = 3.5
                    EFi_NOx = 7.5
                if V < 85:
                    EFi_PM10 = 0.00030
                    EFi_PM2_5 = k * 0.00030
                else:
                    EFi_PM10 = 0.00075
                    EFi_PM2_5 = k * 0.00075
    elif fuel == '경유':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.7392 * (V**-0.7524)
                EFi_HC = 0.0989 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0839 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0839 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5775 * (V**-0.7524)
                EFi_HC = 0.0825 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0420 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0420 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '중형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.6930 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.5414 * (V**-0.7524)
                EFi_HC = 0.0927 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0396 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0396 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승용') and (car_size == '대형'):
            if car_birth <= 2004:
                EFi_CO = 5.9672 * (V**-0.9534)
                EFi_HC = 0.6523 * (V**-1.0167)
                EFi_NOx = 24.3491 * (V**-0.7277)
                if V <= 65:
                    EFi_PM10 = 0.3861 * (V**-0.5093)
                    EFi_PM2_5 = k * 0.3861 * (V**-0.5093)
                else:
                    EFi_PM10 = -0.00001 * (V**2) + 0.0026 * V - 0.0618
                    EFi_PM2_5 = k * -0.00001 * (V**2) + 0.0026 * V - 0.0618
            elif car_birth == 2005:
                EFi_CO = 0.9609 * (V**-0.7524)
                EFi_HC = 0.1865 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0723 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0723 * (V**-0.3420)
            elif 2006 <= car_birth <= 2010:
                EFi_CO = 0.7507 * (V**-0.7524)
                EFi_HC = 0.1554 * (V**-0.6848)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.0361 * (V**-0.3420)
                EFi_PM2_5 = k * 0.0361 * (V**-0.3420)
            elif 2011 <= car_birth <= 2015:
                EFi_CO = 0.5141 * (V**-0.6792)
                EFi_HC = 0.3713 * (V**-0.7513)
                EFi_NOx = 0.0003 * (V**2) - 0.0324 * V + 1.4773
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
            elif 2016 <= car_birth:
                EFi_CO = 0.4574 * (V**-0.5215)
                EFi_HC = 0.1300 * (V**-0.7265)
                EFi_NOx = 2.7702 * (V**-0.3869)
                if V <= 65.4:
                    EFi_PM10 = 0.0225 * (V**-0.7264)
                    EFi_PM2_5 = k * 0.0225 * (V**-0.7264)
                else:
                    EFi_PM10 = 0.0009 * (V**-0.0416)
                    EFi_PM2_5 = k * 0.0009 * (V**-0.0416)
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 0.9835 * (V**-0.5096)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 1.1412 * (V**-0.4324)
                EFi_PM2_5 = k * 1.1412 * (V**-0.4324)
            elif 1991 <= car_birth <= 1995:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.6313 * (V**-0.7298)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5999 * (V**-0.3294)
                EFi_PM2_5 = k * 0.5999 * (V**-0.3294)
            elif 1996 <= car_birth <= 1997:
                if V <= 65.4:
                    EFi_CO = 3.4539 * (V**-0.4266)
                else:
                    EFi_CO = 0.0051 * V + 0.2212
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6408 * (V**-0.3596)
                EFi_PM2_5 = k * 0.6408 * (V**-0.3596)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.5168 * (V**-0.3596)
                EFi_PM2_5 = k * 0.5168 * (V**-0.3596)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 3.7564 * (V**-0.5175)
                EFi_HC = 1.1293 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2894 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2894 * (V**-0.3596)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.011666 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                if V <= 65.4:
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_NOx = 0.0271 * (V**0.7596)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif (car_type == '승합') and (car_size == '중형'):
            if car_birth <= 1995:
                EFi_CO = 32.550 * (V**-0.4944)
                EFi_HC = 15.753 * (V**-0.5912)
                if V < 80:
                    EFi_NOx = 40.692 * (V**-0.5590)
                else:
                    EFi_NOx = -0.0023 * (V**2) + 0.5381 * V - 23.590
                EFi_PM10 = 5.4886 * (V**-0.5911)
                EFi_PM2_5 = k * 5.4886 * (V**-0.5911)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 22.804 * (V**-0.4660)
                else:
                    EFi_NOx = -0.0021 * (V**2) + 0.4430 * V - 18.730
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 16.410 * (V**-0.3790)
                EFi_HC = 4.2324 * (V**-0.3926)
                if V < 80:
                    EFi_NOx = 25.708 * (V**-0.4772)
                else:
                    EFi_NOx = 0.0019 * (V**2) - 0.2628 * V + 12.145
                EFi_PM10 = 1.6593 * (V**-0.3935)
                EFi_PM2_5 = k * 1.6593 * (V**-0.3935)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 16.378 * (V**-0.5340)
                EFi_HC = 5.8477 * (V**-0.5466)
                if V < 80:
                    EFi_NOx = 25.436 * (V**-0.4656)
                else:
                    EFi_NOx = 0.0008 * (V**2) - 0.0482 * V + 1.8424
                EFi_PM10 = 1.2848 * (V**-0.4715)
                EFi_PM2_5 = k * 1.2848 * (V**-0.4715)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                EFi_NOx = 15.001 * (V**-0.4528)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 1.0457 * (V**-0.4527)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 2.0000*(10**-4) * V - 7.5600*(10**-3)
                    EFi_PM2_5 = k * 2.0000*(10**-4) * V - 7.5600*(10**-3)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 1.6800*(10**-4) * V - 5.1600*(10**-3)
                    EFi_PM2_5 = k * 1.6800*(10**-4) * V - 5.1600*(10**-3)
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000*(10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000*(10**-5) * V - 0.0011
        elif (car_type == '승합') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 28.205 * (V**-0.5337)
                EFi_HC = 6.1146 * (V**-0.4979)
                EFi_NOx = 41.346 * (V**-0.3645)
                EFi_PM10 = 5.2158 * (V**-0.5048)
                EFi_PM2_5 = k * 5.2158 * (V**-0.5048)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 2.4911 * (V**-0.4149)
                EFi_PM2_5 = k * 2.4911 * (V**-0.4149)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 23.205 * (V**-0.5425)
                EFi_HC = 6.5657 * (V**-0.5431)
                EFi_NOx = 42.1379 * (V**-0.3786)
                EFi_PM10 = 1.4432 * (V**-0.3870)
                EFi_PM2_5 = k * 1.4432 * (V**-0.3870)
            elif car_birth == 2001:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 0.9375 * (V**-0.3910)
                EFi_PM2_5 = k * 0.9375 * (V**-0.3910)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 21.348 * (V**-0.5806)
                EFi_HC = 6.6390 * (V**-0.5760)
                EFi_NOx = 36.7191 * (V**-0.3548)
                EFi_PM10 = 1.1507 * (V**-0.4804)
                EFi_PM2_5 = k * 1.1507 * (V**-0.4804)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 9.6452 * (V**-0.5291)
                EFi_HC = 3.2339 * (V**-0.7436)
                EFi_NOx = 30.5870 * (V**-0.3548)
                if V <= 80:
                    EFi_PM10 = 0.4657 * (V**-0.5634)
                    EFi_PM2_5 = k * 0.4657 * (V**-0.5634)
                else:
                    EFi_PM10 = 0.0014 * (V**0.7970)
                    EFi_PM2_5 = k * 0.0014 * (V**0.7970)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 11.4415 * (V**-0.8036)
                EFi_HC = 0.6774 * (V**-0.8321)
                EFi_NOx = 112.1229 * (V**-1.6393)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
        elif (car_type == '화물') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 4.5854 * (V**-0.3613)
                EFi_HC = 0.4840 * (V**-0.2756)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.8117 * (V**-0.4071)
                EFi_PM2_5 = k * 0.8117 * (V**-0.4071)
            elif 1991 <= car_birth <= 1995:
                EFi_CO = 3.4774 * (V**-0.3483)
                EFi_HC = 0.4844 * (V**-0.3288)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6188 * (V**-0.4540)
                EFi_PM2_5 = k * 0.6188 * (V**-0.4540)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 3.3934 * (V**-0.3837)
                EFi_HC = 0.4955 * (V**-0.3393)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.7037 * (V**-0.5357)
                EFi_PM2_5 = k * 0.7037 * (V**-0.5357)
            elif 1998 <= car_birth <= 1999:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.6157 * (V**-0.5357)
                EFi_PM2_5 = k * 0.6157 * (V**-0.5357)
            elif 2000 <= car_birth <= 2003:
                EFi_CO = 4.0896 * (V**-0.6083)
                EFi_HC = 0.6122 * (V**-0.5684)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.4838 * (V**-0.5357)
                EFi_PM2_5 = k * 0.4838 * (V**-0.5357)
            elif 2004 <= car_birth <= 2007:
                EFi_CO = 3.2797 * (V**-0.8887)
                EFi_HC = 0.1807 * (V**-0.6588)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.2067 * (V**-0.3596)
                EFi_PM2_5 = k * 0.2067 * (V**-0.3596)
            elif 2008 <= car_birth <= 2011:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_NOx = 24.3491 * (V**-0.7277)
                EFi_PM10 = 0.3111 * (V**-0.5125)
                EFi_PM2_5 = k * 0.3111 * (V**-0.5125)
            elif 2012 <= car_birth <= 2016:
                if V <= 65.4:
                    EFi_CO = 4.222 * (V**-1.4035)
                    EFi_NOx = 2.0217 * (V**-0.2645)
                else:
                    EFi_CO = 0.01166 * (V**0.09222)
                    EFi_NOx = 0.0271 * (V**0.7596)
                if V <= 97.3:
                    EFi_HC = 0.829 * (V**-1.0961)
                EFi_PM10 = 0.1119 * (V**-0.5125)
                EFi_PM2_5 = k * 0.1119 * (V**-0.5125)
        elif ( (car_type == '화물') and (car_size == '중형') ) | (car_type == '특수'):
            if car_birth <= 1995:
                EFi_CO = 16.769 * (V**-0.3772)
                EFi_HC = 6.7755 * (V**-0.5003)
                EFi_NOx = 24.915 * (V**-0.3942)
                EFi_PM10 = 3.6772 * (V**-0.5514)
                EFi_PM2_5 = k * 3.6772 * (V**-0.5514)
            elif 1996 <= car_birth <= 2000:
                EFi_CO = 21.057 * (V**-0.4958)
                EFi_HC = 6.7532 * (V**-0.5711)
                EFi_NOx = 25.022 * (V**-0.4240)
                EFi_PM10 = 3.5285 * (V**-0.5962)
                EFi_PM2_5 = k * 3.5285 * (V**-0.5962)
            elif car_birth == 2001:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.4444 * (V**-0.4824)
                EFi_PM2_5 = k * 1.4444 * (V**-0.4824)
            elif 2002 <= car_birth <= 2004:
                EFi_CO = 25.501 * (V**-0.6100)
                EFi_HC = 6.8738 * (V**-0.5913)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 1.0432 * (V**-0.4992)
                EFi_PM2_5 = k * 1.0432 * (V**-0.4992)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 15.256 * (V**-0.7448)
                EFi_HC = 2.0502 * (V**-0.6504)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                EFi_PM10 = 0.2979 * (V**-0.4008)
                EFi_PM2_5 = k * 0.2979 * (V**-0.4008)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 8.1771 * (V**-0.7725)
                EFi_HC = 1.2991 * (V**-0.6538)
                if V <= 64.7:
                    EFi_NOx = 17.3032 * (V**-0.3660)
                    EFi_PM10 = 0.0539 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0539 * (V**-0.5182)
                else:
                    EFi_NOx = 0.3259 * (V**0.5773)
                    EFi_PM10 = 0.0002 * V - 0.00756
                    EFi_PM2_5 = k * 0.0002 * V - 0.00756
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 4.5201 * (V**-0.7279)
                EFi_HC = 1.6826 * (V**-0.8045)
                if V <= 64.7:
                    EFi_NOx = 17.2485 * (V**-0.4040)
                    EFi_PM10 = 0.0469 * (V**-0.4674)
                    EFi_PM2_5 = k * 0.0469 * (V**-0.4674)
                else:
                    EFi_NOx = 1.1797 * (V**0.2308)
                    EFi_PM10 = 0.000168 * V - 0.00516
                    EFi_PM2_5 = k * 0.000168 * V - 0.00516
            elif 2015 <= car_birth:
                EFi_CO = 7.4065 * (V**-0.5995)
                EFi_HC = 2.4562 * (V**-1.3145)
                EFi_NOx = 42.7393 * (V**-1.2949)
                if V <= 64.7:
                    EFi_PM10 = 0.0081 * (V**-0.5182)
                    EFi_PM2_5 = k * 0.0081 * (V**-0.5182)
                else:
                    EFi_PM10 = 3.0000 * (10**-5) * V - 0.0011
                    EFi_PM2_5 = k * 3.0000 * (10**-5) * V - 0.0011
        elif (car_type == '화물') and (car_size == '대형'):
            if car_birth <= 1995:
                EFi_CO = 30.402 * (V**-0.4685)
                EFi_HC = 15.75 * (V**-0.582)
                EFi_NOx = 117.49 * (V**-0.365)
                EFi_PM10 = 7.6212 * (V**-0.4183)
                EFi_PM2_5 = k * 7.6212 * (V**-0.4183)
            elif 1996 <= car_birth <= 1997:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 6.0264 * (V**-0.4627)
                EFi_PM2_5 = k * 6.0264 * (V**-0.4627)
            elif 1998 <= car_birth <= 2000:
                EFi_CO = 18.101 * (V**-0.3454)
                EFi_HC = 10.301 * (V**-0.5856)
                EFi_NOx = 94.319 * (V**-0.4061)
                EFi_PM10 = 4.873 * (V**-0.4382)
                EFi_PM2_5 = k * 4.873 * (V**-0.4382)
            elif 2001 <= car_birth <= 2004:
                EFi_CO = 28.399 * (V**-0.5999)
                EFi_HC = 10.031 * (V**-0.5828)
                EFi_NOx = 85.301 * (V**-0.4023)
                EFi_PM10 = 3.7541 * (V**-0.4055)
                EFi_PM2_5 = k * 3.7541 * (V**-0.4055)
            elif 2005 <= car_birth <= 2007:
                EFi_CO = 52.136 * (V**-0.8618)
                EFi_HC = 3.7878 * (V**-0.5425)
                EFi_NOx = 107.5 * (V**-0.5679)
                EFi_PM10 = 2.6847 * (V**-0.6112)
                EFi_PM2_5 = k * 2.6847 * (V**-0.6112)
            elif 2008 <= car_birth <= 2010:
                EFi_CO = 6.8493 * (V**-0.6506)
                EFi_HC = 1.7177 * (V**-0.6781)
                EFi_NOx = 40.7564 * (V**-0.4757)
                EFi_PM10 = 0.2418 * (V**-0.4727)
                EFi_PM2_5 = k * 0.2418 * (V**-0.4727)
            elif 2011 <= car_birth <= 2014:
                EFi_CO = 5.4607 * (V**-0.2990)
                EFi_HC = 0.8863 * (V**-0.6933)
                EFi_NOx = 40.3729 * (V**-0.5386)
                EFi_PM10 = 0.2125 * (V**-0.4650)
                EFi_PM2_5 = k * 0.2125 * (V**-0.4650)
            elif 2015 <= car_birth:
                EFi_CO = 4.3762 * (V**-0.4550)
                EFi_HC = 0.3627 * (V**-0.7071)
                EFi_NOx = 18.0405 * (V**-1.0986)
                EFi_PM10 = 0.0363 * (V**-0.4727)
                EFi_PM2_5 = k * 0.0363 * (V**-0.4727)
    elif fuel == 'LPG(액화석유가스)':
        if (car_type == '승용') and (car_size == '경형'):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승용') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 101.79 * (V**-1.6823)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 7.5371 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 11.173 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                EFi_CO = 44.956 * (V**-1.0085)
                EFi_HC = 3.2821 * (V**-1.3927)
                EFi_NOx = 4.7108 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 39.362 * (V**-1.0085)
                EFi_HC = 2.8981 * (V**-1.3927)
                EFi_NOx = 1.8419 * (V**-0.7864)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승용') and ( (car_size == '중형') or (car_size == '대형') ) or ( (car_type == '특수') and (car_size == '중형') ) ): # 최 확인(2023.04.20) : 특수 중형 -> 승용 중형 조건으로 계산
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth <= 2007:
                EFi_CO = 73.022 * (V**-1.2078)
                EFi_HC = 4.4166 * (V**-1.5356)
                EFi_NOx = 2.0280 * (V**-0.7978)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2008:
                if V <= 79.6:
                    EFi_CO = 0.7693 * (V**-0.7666)
                    EFi_HC = 0.1063 * (V**-1.0745)
                else:
                    EFi_CO = 5.0000 * (10**-16) * (V**7.2766)
                    EFi_HC = 1.0000 * (10**-15) * (V**6.2696)
                EFi_NOx = -4.0000 * (10**-6) * (V**2) + 6.0000 * (10**-4) * V + 5.5000 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2009 <= car_birth <= 2011:
                if V <= 79.6:
                    EFi_CO = 0.7059 * (V**-0.7666)
                    EFi_HC = 0.0974 * (V**-1.0745)
                else:
                    EFi_CO = 4.5878 * (10**-16) * (V**7.2766)
                    EFi_HC = 9.1667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.7333 * (10**-6) * (V**2) + 5.6000 * (10**-4) * V + 5.1333 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2012 <= car_birth <= 2013:
                if V <= 79.6:
                    EFi_CO = 0.6830 * (V**-0.7666)
                    EFi_HC = 0.0943 * (V**-1.0745)
                else:
                    EFi_CO = 4.4393 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.8667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.6373 * (10**-6) * (V**2) + 5.4560 * (10**-4) * V + 5.0013 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif car_birth == 2014:
                if V <= 79.6:
                    EFi_CO = 0.6602 * (V**-0.7666)
                    EFi_HC = 0.0911 * (V**-1.0745)
                else:
                    EFi_CO = 4.2909 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.5667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.5413 * (10**-6) * (V**2) + 5.3120 * (10**-4) * V + 4.8693 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2015 <= car_birth:
                if V <= 79.6:
                    EFi_CO = 0.6374 * (V**-0.7666)
                    EFi_HC = 0.0879 * (V**-1.0745)
                else:
                    EFi_CO = 4.1425 * (10**-16) * (V**7.2766)
                    EFi_HC = 8.2667 * (10**-16) * (V**6.2696)
                EFi_NOx = -3.4453 * (10**-6) * (V**2) + 5.1680 * (10**-4) * V + 4.7373 * (10**-3)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif ( (car_type == '승합') and (car_size == '경형') ) or ( (car_type == '화물') and (car_size == '경형') ):
            if car_birth <= 1996:
                if V <= 45:
                    EFi_CO = 22.498 * (V**-0.6579)
                else:
                    EFi_CO = 0.0006 * (V**2) + 0.0004 * V + 0.8272
                EFi_HC = 12.961 * (V**-0.8364)
                EFi_NOx = 4.0131 * (V**-0.3236)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth == 1999:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 1.8528 * (V**-0.3889)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2002:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 2.2714 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2003 <= car_birth <= 2005:
                if V <= 45:
                    EFi_CO = 19.887 * (V**-0.8461)
                else:
                    EFi_CO = -0.0004 * (V**2) + 0.0911 * V - 2.2698
                EFi_HC = 1.1073 * (V**-0.7830)
                EFi_NOx = 5.8289 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 45:
                    EFi_CO = 8.9904 * (V**-0.8461)
                else:
                    EFi_CO = -0.0002 * (V**2) + 0.0457 * V - 1.1403
                EFi_HC = 0.3549 * (V**-0.7830)
                EFi_NOx = 0.7228 * (V**-0.9198)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        elif (car_type == '승합') and (car_size == '소형'):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 6.1701 * (V**-1.0719)
                    EFi_HC = 2.3782 * (V**-1.9979)
                else:
                    EFi_CO = 3.0000 * (10**-10) * (V**4.5809)
                    EFi_HC = 5.0000 * (10**-10) * (V**3.4895)
                if V <= 97.3:
                    EFi_NOx = 0.4758 * (V**-1.0665)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
        # 최 확인(2023.04.20) : 화물 중형 -> 승합 중형 조건, 화물 대형 -> 승합 대형 조건으로 계산
        elif ( (car_type == '승합') and ( (car_size == '중형') or (car_size == '대형') ) ) or ( (car_type == '화물') and ((car_size == '소형') or (car_size == '중형') or (car_size == '대형')) ):
            if car_birth <= 1990:
                EFi_CO = 72.338 * (V**-0.7587)
                EFi_HC = 26.532 * (V**-1.0423)
                EFi_NOx = 9.3011 * (V**-0.6781)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1991 <= car_birth == 1996:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 26.520 * (V**-1.4041)
                EFi_NOx = 11.287 * (V**-0.7634)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 1997 <= car_birth <= 1999:
                EFi_CO = 29.825 * (V**-0.6771)
                EFi_HC = 4.7595 * (V**-0.9512)
                EFi_NOx = 12.562 * (V**-0.8606)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2000 <= car_birth <= 2005:
                EFi_CO = 17.829 * (V**-0.6778)
                EFi_HC = 6.3668 * (V**-1.2849)
                EFi_NOx = 8.8952 * (V**-0.8119)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
            elif 2006 <= car_birth:
                if V <= 65.4:
                    EFi_CO = 16.491 * (V**-1.4713)
                    EFi_HC = 1.9218 * (V**-1.8418)
                else:
                    EFi_CO = 3.0000 * (10**-13) * (V**6.0619)
                    EFi_HC = 3.0000 * (10**-9) * (V**3.0639)
                if V <= 97.3:
                    EFi_NOx = 0.1246 * (V**-0.441)
                if V < 85:
                    EFi_PM10 = 0.0002
                    EFi_PM2_5 = k * 0.0002
                else:
                    EFi_PM10 = 0.0005
                    EFi_PM2_5 = k * 0.0005
    else:
        # 값 고민 : 1
        # EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = np.nan, np.nan, np.nan, np.nan, np.nan
        EFi_CO, EFi_HC, EFi_NOx, EFi_PM10, EFi_PM2_5 = 1, 1, 1, 1, 1

    # 배출계수(EFi) 리스트에 저장
    EFi_CO_list.append(EFi_CO)
    EFi_HC_list.append(EFi_HC)
    EFi_NOx_list.append(EFi_NOx)
    EFi_PM10_list.append(EFi_PM10)
    EFi_PM2_5_list.append(EFi_PM2_5)

100%|██████████| 1497708/1497708 [00:06<00:00, 244566.86it/s]


In [124]:
df2.shape[0]

1497708

In [125]:
len(EFi_CO_list), len(EFi_HC_list), len(EFi_NOx_list), len(EFi_PM10_list), len(EFi_PM2_5_list)

(1497708, 1497708, 1497708, 1497708, 1497708)

In [126]:
df2['EFi_CO'] = EFi_CO_list
df2['EFi_HC'] = EFi_HC_list
df2['EFi_NOx'] = EFi_NOx_list
df2['EFi_PM10'] = EFi_PM10_list
df2['EFi_PM2_5'] = EFi_PM2_5_list

In [127]:
check_EFi_col = ['EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5']

## 배출량 계산

In [128]:
DF_col + check_DF_col + check_R_col

['연료',
 '차종',
 '차종유형',
 '연식',
 '차령',
 'DF_CO',
 'DF_HC',
 'DF_PM',
 'DF_NOx',
 'R_CO',
 'R_HC',
 'R_PM']

In [129]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,R_HC,R_PM,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5
0,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.04,0.01,1.33,0.01,0.01
1,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.04,0.01,1.33,0.01,0.01
2,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.04,0.01,1.33,0.01,0.01
3,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.04,0.01,1.33,0.01,0.01
4,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,0.00,0.00,0.04,0.01,1.33,0.01,0.01


In [130]:
# E = VKT * (EFi / 1000) * DF * (1 - R / 100)
df2['E_HOT_CO'] = df2['VKT'] * (df2['EFi_CO'] / 1000) * df2['DF_CO'] * (1 - df2['R_CO'] / 100)
df2['E_HOT_HC'] = df2['VKT'] * (df2['EFi_HC'] / 1000) * df2['DF_HC'] * (1 - df2['R_HC'] / 100)
df2['E_HOT_NOx'] = df2['VKT'] * (df2['EFi_NOx'] / 1000) * df2['DF_NOx'] * (1 - 0 / 100)
df2['E_HOT_PM10'] = df2['VKT'] * (df2['EFi_PM10'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)
df2['E_HOT_PM2_5'] = df2['VKT'] * (df2['EFi_PM2_5'] / 1000) * df2['DF_PM'] * (1 - df2['R_PM'] / 100)

In [131]:
check_E_HOT_col = ['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5']

In [132]:
df2[DF_col + check_DF_col + check_R_col + check_EFi_col + check_E_HOT_col].head()

,연료,차종,차종유형,연식,차령,DF_CO,DF_HC,DF_PM,DF_NOx,R_CO,...,EFi_CO,EFi_HC,EFi_NOx,EFi_PM10,EFi_PM2_5,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5
0,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,...,0.04,0.01,1.33,0.01,0.01,0.73,0.20,20.80,0.18,0.17
1,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,...,0.04,0.01,1.33,0.01,0.01,0.74,0.20,21.16,0.18,0.17
2,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,...,0.04,0.01,1.33,0.01,0.01,0.76,0.20,21.62,0.19,0.17
3,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,...,0.04,0.01,1.33,0.01,0.01,0.76,0.20,21.62,0.19,0.17
4,경유,승용,대형,2006,17,1.50,1.50,1.50,1.20,0.00,...,0.04,0.01,1.33,0.01,0.01,0.74,0.20,21.10,0.18,0.17


In [133]:
df2.shape

(1497708, 58)

# 자동차-엔진 미가열(Cold-start) 배출

## 배출량 식
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$

- $E_{COLD :i, j}$ : 차종 $j$에서 배출되는 오염물질 $i$의 엔진미가열 배출량
- $\beta_{i, j}$ : 차종 $j$의 엔진미가열 상태의 주행거리 분율
- $N_j$ : 차종 $j$의 수
- $M_j$ : 차종 $j$의 주행거리
- $e^{HOT}_j$ : 차종 $j$의 엔진가열 상태에서의 배출계수
- $e^{COLD} / e^{HOT} \vert_{i, j}$ : 차종 $j$의 엔진가열상태 대비 엔진미가열 상태에서의 배출 비율

## 베타($\beta$)
- 1회 평균주행거리(1 trip length)와 대기온도, 자동차 이용 패턴을 고려
    - 1회 평균주행거리 정보는 모든 차종에 대하여 각각 적용하는 방안은 수집자료의 한계로 국내 연구결과를 바탕으로 수도권 지역 승용차의 1회 평균 주행거리 12.35km를 적용
    - 이때 과대산정 가능성을 염두하여, 도시지역의 택시를 제외한 승용차(경형, 소형, 중형, 대형), 승합차(경형, 소형), RV(소형, 중형)에 적용

    - 1회 주행거리 : 12.35 km (2002년 기준)
    - 미가열 배출 적용 차종 : 승용차(경형, 소형, 중형, 대형) / 승합차(경형, 소형) / RV(소형, 중형)
    - 미가열 배출 적용 지역 : 도시지역(고속도로 구간 제외)

- 산정식

-|Factor $\beta$의 산정식
-|-
$Estimated$ $l_{trip}$|$0.647-0.025 \times l_{trip} - (0.00974 - 0.000385 \times l_{trip}) \times T_a$
$Measured$ $l_{trip}$|$0.698-0.051 \times l_{trip} - (0.01051 - 0.000770 \times l_{trip}) \times T_a$

- $T_a$ : 대기온도
- $Estimated$ $l_{trip}$ : 유럽평균(1985) 약 11.8km / 유럽평균(1998) 약 12.4km(Andre et al., 1998)

In [134]:
l_trip = 12.35 # 국립환경과학원(2007), 도로 이동오염원 대기오염 배출량 산정방법 개선 및 장래 배출량 예측방법 연구
Ta = 12.4 # e-나라지표 (https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1400) 2002년 년 평균 기온

In [135]:
df2['Beta(Estimated)'] = 0.647 - 0.025 * l_trip - (0.00974 - 0.000385 * l_trip) * Ta
# df2['Beta(Measured)'] = 0.698 - 0.051 * l_trip - (0.01051 - 0.000770 * l_trip) * Ta

In [136]:
df2['Beta(Estimated)'].head()

0   0.28
1   0.28
2   0.28
3   0.28
4   0.28
Name: Beta(Estimated), dtype: float64

In [137]:
df2['Beta(Estimated)'].isnull().sum()

0

## $e^{COLD} / e^{HOT}$

In [138]:
df2['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [139]:
df2['연식'].describe()

count   1497708.00
mean       2005.87
std           6.83
min        1901.00
25%        2006.00
50%        2009.00
75%        2010.00
max        2018.00
Name: 연식, dtype: float64

In [140]:
df2['제작일자'].describe()

count    1330654.00
mean    20072621.19
std        52791.07
min         1993.00
25%     20070917.00
50%     20090406.00
75%     20100616.00
max     20190923.00
Name: 제작일자, dtype: float64

In [141]:
# 휘발유(자동 제어)
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 3.7 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_CO'] = 9.04 - 0.09 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 1.14 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_NOx'] = 3.66 - 0.006 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] < 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 2.8 - 0.06 * Ta
df2.loc[(df2['연료'] == '휘발유') & (df2['연식'] >= 1991) & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ), 'eCOLD/eHOT_HC'] = 12.59 - 0.06 * Ta
df2.loc[df2['연료'] == '휘발유', 'eCOLD/eHOT_PM'] = 1
# 경유
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 1.9 - 0.03 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 1.3 - 0.013 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 3.1 - 0.09 * Ta
df2.loc[(df2['연료'] == '경유') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 3.1 - 0.1 * Ta
# LPG(액화석유가스)
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_CO'] = 3.66 - 0.09 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_NOx'] = 0.98 - 0.006 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_HC'] = 2.24 - 0.06 * Ta
df2.loc[(df2['연료'] == 'LPG(액화석유가스)') & ( (df2['차종'] == '승용') | ( (df2['차종'] == '승합') & ((df2['차종유형'] == '경형') | (df2['차종유형'] == '소형')) ) ),'eCOLD/eHOT_PM'] = 1

In [142]:
check_ecoldhot_col = ['eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     640364
eCOLD/eHOT_NOx    640364
eCOLD/eHOT_HC     640364
eCOLD/eHOT_PM     634895
dtype: int64

In [143]:
df2[check_ecoldhot_col] = df2[check_ecoldhot_col].fillna(1)
df2[check_ecoldhot_col].isnull().sum()

eCOLD/eHOT_CO     0
eCOLD/eHOT_NOx    0
eCOLD/eHOT_HC     0
eCOLD/eHOT_PM     0
dtype: int64

In [144]:
df2.loc[df2['eCOLD/eHOT_CO'] < 0, 'eCOLD/eHOT_CO'].shape

(0,)

In [145]:
df2.loc[df2['eCOLD/eHOT_HC'] < 0, 'eCOLD/eHOT_HC'].shape

(0,)

In [146]:
df2.loc[df2['eCOLD/eHOT_PM'] < 0, 'eCOLD/eHOT_PM'].shape

(0,)

## 배출량 계산

In [147]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '1일평균주행거리', 'VKT', 'DF_CO',
       'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx',
       'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM'],
      dtype='object')

### 🔺 계산식 고민
$$E_{COLD :i, j} = \beta_{i, j} \times N_j \times M_j \times e^{HOT}_j \times (e^{COLD} / e^{HOT} \vert_{i, j} - 1)$$
- $M_j$ : 차종 $j$의 주행거리
    - 1년 주행거리로 계산? 
        - VKT
    - 1회 주행거리로 계산?
        - l_trip

In [148]:
#         E      =             B          *     M      *    e(HOT)     * ( e(COLD)/e(HOT)      - 1 )
df2['E_COLD_CO'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_CO'] * (df2['eCOLD/eHOT_CO'] - 1)
df2['E_COLD_NOx'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_NOx'] * (df2['eCOLD/eHOT_NOx'] - 1)
df2['E_COLD_HC'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_HC'] * (df2['eCOLD/eHOT_HC'] - 1)
df2['E_COLD_PM10'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM10'] * (df2['eCOLD/eHOT_PM'] - 1)
df2['E_COLD_PM2_5'] = df2['Beta(Estimated)'] * df2['VKT'] * df2['EFi_PM2_5'] * (df2['eCOLD/eHOT_PM'] - 1)

In [149]:
df2.loc[df2['eCOLD/eHOT_CO'] - 1 < 0, ['연료', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM']].head()

,연료,eCOLD/eHOT_CO,eCOLD/eHOT_NOx,eCOLD/eHOT_HC,eCOLD/eHOT_PM


In [150]:
check_E_cold_col = ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5']
df2[check_E_HOT_col + check_E_cold_col].head()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
0,0.73,0.20,20.80,0.18,0.17,70.69,665.21,35.72,28.48,26.20
1,0.74,0.20,21.16,0.18,0.17,71.88,676.42,36.32,28.96,26.65
2,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23
3,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23
4,0.74,0.20,21.10,0.18,0.17,71.69,674.55,36.22,28.88,26.57


In [151]:
df2[check_E_HOT_col + check_E_cold_col].tail()

,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5
1497703,48.45,6.74,11.46,0.01,0.01,10608.08,103.89,984.00,0.00,0.00
1497704,47.05,6.55,11.13,0.01,0.01,10300.19,100.88,955.44,0.00,0.00
1497705,47.81,6.65,11.31,0.01,0.01,10468.13,102.52,971.02,0.00,0.00
1497706,46.66,6.49,11.04,0.01,0.01,10216.22,100.06,947.65,0.00,0.00
1497707,46.28,6.44,10.94,0.01,0.01,10132.25,99.23,939.86,0.00,0.00


In [152]:
df2[check_E_cold_col].isnull().sum()

E_COLD_CO       0
E_COLD_NOx      0
E_COLD_HC       0
E_COLD_PM10     0
E_COLD_PM2_5    0
dtype: int64

# 자동차-휘발유 증발 배출

## 배출량 식
- 3가지
    - 주간증발손실(Diurnal loss)
    - 고온증발손실(Hot and warm soak)
    - 주행손실(Running loss)
- 현재('19 배출량 기준) 휘발유 증발 배출원
    - 주간증발손실, 주행손실 고려
    - 우리나라 휘발유 차량 대부분이 방지설비로 카본 캐니스터를 장착하고 있으므로 $e^{S,HOT}$, $e^{S,WARM}$, 고온증발손실(hot and warm soak) 배출은 없는 것으로 가정

---

$$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$

$S^{fi} = q \times e^{fi} \times x$
$$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$$

- $E_{EVA, VOC:j}$ : 차종 j의 증발손실에 의한 VOC 배출량(g/yr)
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수
- $m_j$ : 차종 j의 연간 총 주행거리
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

## 배출계수

### 🔺 수정 중

### $e^d$ Diurnal(g/day)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)

#### 🔺 RVP(증기압)
- 참고 COPERT 3(p.80) Table 6.10: Gasoline fuel specifications
    - summer : 60
    - winter : 70
    - 평균 : 65
- 수정 사항(2023.04.20, 최)
    - 국내 과학기술원, 환경부 자료 찾아서 사용 추천

In [153]:
RVP = 65

#### ❗ t_a(기온)
- t_a, t_min, t_rise
    - 참고 COPERT 3(p.72) Table 5.32: Summary of emission factors for estimating evaporative emissions of gasoline vehicles(all RVp in kPa, all temperatures in °C)
        - t_a = (t_max + t_min) / 2
        - t_rise = t_max - t_min
    - 기상자료개방포털(2002년) 연평균 기온
        - https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
        - 7.5°C(최저), 12.4°C(평균), 17.9°C(최고)

In [154]:
t_min = 7.5 # 평균 최저 온도
t_max = 17.9
t_a = (t_min + t_max) / 2
t_rise = 12.4 # 평균 상승 온도

In [155]:
df2.loc[df2['연식'] < 1991, 'e_d'] = 9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) )
df2.loc[df2['연식'] >= 1991, 'e_d'] = 0.2 * (9.1 * np.exp( 0.0158 * (RVP - 61.2) + 0.0574 * (t_min - 22.5) + 0.0614 * (t_rise - 11.7) ))

In [156]:
df2['e_d'].isnull().sum()

0

### $e^{R,HOT}$ Hot running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [157]:
df2.loc[df2['연식'] < 1991, 'e_RHOT'] = 0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['연식'] >= 1991, 'e_RHOT'] = 0.1 * (0.136 * np.exp( - 5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [158]:
df2['e_RHOT'].isnull().sum()

0

### $e^{S,HOT}$ Hot soak(g/procedure)
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수

In [159]:
df2.loc[df2['연식'] < 1991, 'e_SHOT'] = 3.0042 * np.exp(0.02 * RVP)
df2.loc[df2['연식'] >= 1991, 'e_SHOT'] = 0.3 * np.exp(-2.14 + 0.02302 * RVP + 0.09408 * t_a)

### $e^{S,WARM}$ Waram soak(g/procedure)
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [160]:
df2.loc[df2['연식'] < 1991, 'e_SWARM'] = np.exp(-1.644 + 0.01993 * RVP + 0.07521 * t_a)
df2.loc[df2['연식'] >= 1991, 'e_SWARM'] = 0.2 * np.exp(-2.41 + 0.02302 * RVP + 0.09408 * t_a)

### $e^{fi}$ Warm and hot soak for Fuel injected vehicle(g/procedure)
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수

In [161]:
df2.loc[df2['연식'] < 1991, 'e_fi'] = 0.7
df2.loc[df2['연식'] >= 1991, 'e_fi'] = 0

### $e^{R,WARM}$ Warm running loss(g/km)
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수

In [162]:
df2.loc[df2['연식'] < 1991, 'e_RWARM'] = 0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a)
df2.loc[df2['연식'] >= 1991, 'e_RWARM'] = 0.1 * (0.1 * np.exp(-5.967 + 0.04259 * RVP + 0.1773 * t_a))

In [163]:
df2['e_RWARM'].isnull().sum()

0

## $R$
$R = m_j(p \times e^{R,HOT} + w \times e^{R,WARM})$
- Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)
- $m_j$ : 차종 j의 연간 총 주행거리
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $e^{R,HOT}$ : 휘발유 차량의 평균 hot running loss 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $e^{R,WARM}$ : 휘발유 차량의 평균 warm running loss 배출계수

### ❓ 확인 중
- 수정 사항(2023.04.20, 최)
    - 찾는 중

In [164]:
p = 0.5 # 가열 엔진 상태로 끝나는 trip의 비율
w = 0.5 #미가열 엔진 상태로 끝나는 trip의 비율

In [165]:
df2['R'] = df2['VKT'] * (p * df2['e_RHOT'] + w * df2['e_RWARM'])

In [166]:
df2['R'].isnull().sum()

0

## $S^{fi}$
$S^{fi} = q \times e^{fi} \times x$
- Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $e^{fi}$ : Fuel injection 휘발유 차량의 평균 hot and warm soak 배출계수
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리

In [167]:
injection = df2[df2['연식'] < 1991].shape[0]
not_injection = df2[df2['연식'] >= 1991].shape[0]
# Fuel injection 휘발유 차량의 비율
q = injection / (injection + not_injection)
q

0.04005520435225024

In [168]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_fi'] = q * df2['e_fi'] * ( df2['VKT'] / (365 * l_trip) )

## $S^c$
$S^c = (1-q)(p \times x \times e^{S, HOT} + w \times x \times e^{S,WARM})$
- Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $q$ : Fuel injection 휘발유 차량의 비율
- $p$ : 가열 엔진 상태로 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $e^{S,HOT}$ : Hot soak emission의 평균 배출계수
- $w$ : 미가열 엔진 상태료 끝나는 trip의 비율
- $x$ : 일평균 통행횟수 $= V_j / 365 \times l_{trip}$
- $V_j$ : 차종 j의 1대당 평균 연간 주행거리
- $e^{S,WARM}$ : Cold and warm soak emission의 평균 배출계수

In [169]:
# x = df2['VKT'] / (365 * l_trip)
df2['S_c'] = (1-q) * (p * (df2['VKT'] / (365 * l_trip)) * df2['e_SHOT'] + w * (df2['VKT'] / (365 * l_trip)) * df2['e_SWARM'])

## $E_{EVA,VOC}$
$E_{EVA, VOC:j} = 365 \times N_j \times (e^d + S^c + S^{fi}) + R$
- $N_j$ : 차종 j의 휘발유 사용 차량 등록대수(대)
- $e^d$ : 금속탱크를 가지고 있는 휘발유 차량의 일중 VOC 배출량(g/day-대)
- $S^c$ : Carburetor 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $S^{fi}$ : Fuel injection 휘발유 차량의 일중 hot and warm soak 배출량(g/day-대)
- $R$ : Running loss에 의한 휘발유 차량의 연중 배출량(g/yr)

In [170]:
df2['E_EVA_VOC'] = 365 * df2['e_d'] + df2['S_c'] + df2['S_fi'] + df2['R']

In [171]:
df2['E_EVA_VOC'].isnull().sum()

0

# 배출량 합계
$E_{total} = E_{HOT} + E_{COLD} + E_{EVAP}$

In [172]:
df2[df2['E_COLD_NOx'] < 0].shape

(14782, 78)

In [173]:
# E_COLD_NOx 음수 -> 0으로 처리(2023.04.24 from 최이사님)
df2.loc[df2['E_COLD_NOx'] < 0, 'E_COLD_NOx'] = 0

In [174]:
check_E_HOT_col, check_E_cold_col

(['E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx', 'E_HOT_PM10', 'E_HOT_PM2_5'],
 ['E_COLD_CO', 'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5'])

In [175]:
df2.loc[(df2['연료'] == '휘발유') & (df2['E_COLD_PM10'] != 0)].shape

(0, 78)

In [176]:
df2['연료'].unique()

array(['경유', '휘발유', 'LPG(액화석유가스)'], dtype=object)

In [177]:
fuel = '휘발유'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC'] + df2.loc[df2['연료'] == fuel , 'E_EVA_VOC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']

In [178]:
fuel = '경유'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']

In [179]:
fuel = 'LPG(액화석유가스)'
df2.loc[df2['연료'] == fuel, 'E_CO_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_CO'] + df2.loc[df2['연료'] == fuel , 'E_COLD_CO']
df2.loc[df2['연료'] == fuel, 'E_HC_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_HC'] + df2.loc[df2['연료'] == fuel , 'E_COLD_HC']
df2.loc[df2['연료'] == fuel, 'E_NOx_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_NOx'] + df2.loc[df2['연료'] == fuel , 'E_COLD_NOx']
df2.loc[df2['연료'] == fuel, 'E_PM10_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM10'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM10']
df2.loc[df2['연료'] == fuel, 'E_PM2_5_total'] = df2.loc[df2['연료'] == fuel , 'E_HOT_PM2_5'] + df2.loc[df2['연료'] == fuel , 'E_COLD_PM2_5']

In [180]:
check_E_col = ['E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']
df2[check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,71.42,35.91,686.01,28.66,26.37
1,72.62,36.52,697.57,29.15,26.81
2,74.23,37.33,712.99,29.79,27.41
3,74.23,37.33,712.99,29.79,27.41
4,72.42,36.42,695.65,29.07,26.74


In [181]:
df2.loc[df2['연료'] == '휘발유', check_E_col + ['E_EVA_VOC']].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total,E_EVA_VOC
1149289,14001.37,1277.35,2349.39,0.01,0.01,368.12
1149290,33430.82,4320.80,3891.35,0.01,0.01,382.07
1149291,31371.83,5360.43,3797.80,0.01,0.01,380.37
1149293,29251.97,3819.61,3404.93,0.01,0.01,373.23
1149294,28769.79,3761.78,3348.80,0.01,0.01,372.21


In [182]:
df2.loc[df2['연료'] == '경유', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,71.42,35.91,686.01,28.66,26.37
1,72.62,36.52,697.57,29.15,26.81
2,74.23,37.33,712.99,29.79,27.41
3,74.23,37.33,712.99,29.79,27.41
4,72.42,36.42,695.65,29.07,26.74


In [183]:
df2.loc[df2['연료'] == 'LPG(액화석유가스)', check_E_col].head()

,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1149292,49.61,1.57,14.49,0.01,0.01
1149359,11144.04,176.64,13.94,0.01,0.00
1149391,14524.82,251.83,13.72,0.01,0.01
1149399,9813.03,442.94,10.82,0.01,0.01
1149429,79.57,4.25,16.11,0.01,0.01


In [184]:
df2[DF_col + check_E_HOT_col + check_E_cold_col + ['E_EVA_VOC'] + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,...,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_EVA_VOC,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,경유,승용,대형,2006,17,0.73,0.20,20.80,0.18,0.17,...,665.21,35.72,28.48,26.20,371.87,71.42,35.91,686.01,28.66,26.37
1,경유,승용,대형,2006,17,0.74,0.20,21.16,0.18,0.17,...,676.42,36.32,28.96,26.65,372.89,72.62,36.52,697.57,29.15,26.81
2,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,...,691.37,37.12,29.60,27.23,374.25,74.23,37.33,712.99,29.79,27.41
3,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,...,691.37,37.12,29.60,27.23,374.25,74.23,37.33,712.99,29.79,27.41
4,경유,승용,대형,2006,17,0.74,0.20,21.10,0.18,0.17,...,674.55,36.22,28.88,26.57,372.72,72.42,36.42,695.65,29.07,26.74


In [185]:
df2[DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,경유,승용,대형,2006,17,0.73,0.20,20.80,0.18,0.17,70.69,665.21,35.72,28.48,26.20,71.42,35.91,686.01,28.66,26.37
1,경유,승용,대형,2006,17,0.74,0.20,21.16,0.18,0.17,71.88,676.42,36.32,28.96,26.65,72.62,36.52,697.57,29.15,26.81
2,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23,74.23,37.33,712.99,29.79,27.41
3,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23,74.23,37.33,712.99,29.79,27.41
4,경유,승용,대형,2006,17,0.74,0.20,21.10,0.18,0.17,71.69,674.55,36.22,28.88,26.57,72.42,36.42,695.65,29.07,26.74


In [186]:
df2[DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].tail()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1497703,휘발유,승용,중형,1989,34,48.45,6.74,11.46,0.01,0.01,10608.08,103.89,984.00,0.00,0.00,10656.53,3199.49,115.35,0.01,0.01
1497704,휘발유,승용,소형,1988,35,47.05,6.55,11.13,0.01,0.01,10300.19,100.88,955.44,0.00,0.00,10347.24,3151.80,112.00,0.01,0.01
1497705,휘발유,승용,중형,1989,34,47.81,6.65,11.31,0.01,0.01,10468.13,102.52,971.02,0.00,0.00,10515.94,3177.82,113.83,0.01,0.01
1497706,휘발유,승용,중형,1989,34,46.66,6.49,11.04,0.01,0.01,10216.22,100.06,947.65,0.00,0.00,10262.89,3138.80,111.09,0.01,0.01
1497707,휘발유,승용,중형,1989,34,46.28,6.44,10.94,0.01,0.01,10132.25,99.23,939.86,0.00,0.00,10178.53,3125.79,110.18,0.01,0.01


In [187]:
df2.loc[(df2['연료'] == '경유') & (df2['차종'] == '승용'), DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
0,경유,승용,대형,2006,17,0.73,0.20,20.80,0.18,0.17,70.69,665.21,35.72,28.48,26.20,71.42,35.91,686.01,28.66,26.37
1,경유,승용,대형,2006,17,0.74,0.20,21.16,0.18,0.17,71.88,676.42,36.32,28.96,26.65,72.62,36.52,697.57,29.15,26.81
2,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23,74.23,37.33,712.99,29.79,27.41
3,경유,승용,대형,2006,17,0.76,0.20,21.62,0.19,0.17,73.47,691.37,37.12,29.60,27.23,74.23,37.33,712.99,29.79,27.41
4,경유,승용,대형,2006,17,0.74,0.20,21.10,0.18,0.17,71.69,674.55,36.22,28.88,26.57,72.42,36.42,695.65,29.07,26.74


In [188]:
df2.loc[(df2['연료'] == '경유') & (df2['차종'] == '화물'), DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
227627,경유,화물,대형,2009,14,0.07,0.33,142.00,0.17,0.16,0.00,0.00,0.00,0.00,0.00,0.07,0.33,142.00,0.17,0.16
227628,경유,화물,대형,2010,13,0.09,0.39,169.84,0.20,0.19,0.00,0.00,0.00,0.00,0.00,0.09,0.39,169.84,0.20,0.19
227629,경유,화물,대형,2009,14,0.08,0.34,146.47,0.18,0.16,0.00,0.00,0.00,0.00,0.00,0.08,0.34,146.47,0.18,0.16
227630,경유,화물,대형,2011,12,0.21,0.14,102.98,0.14,0.13,0.00,0.00,0.00,0.00,0.00,0.21,0.14,102.98,0.14,0.13
227631,경유,화물,대형,2009,14,0.06,0.25,108.08,0.13,0.12,0.00,0.00,0.00,0.00,0.00,0.06,0.25,108.08,0.13,0.12


In [189]:
df2.loc[(df2['연료'] == '휘발유') & (df2['차종'] == '승용'), DF_col + check_E_HOT_col + check_E_cold_col + ['E_EVA_VOC'] + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,...,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_EVA_VOC,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1149289,휘발유,승용,소형,2001,22,14.62,0.61,6.56,0.01,0.01,...,2342.83,908.62,0.00,0.00,368.12,14001.37,1277.35,2349.39,0.01,0.01
1149290,휘발유,승용,대형,1999,24,34.90,2.63,10.86,0.01,0.01,...,3880.49,3936.10,0.00,0.00,382.07,33430.82,4320.80,3891.35,0.01,0.01
1149291,휘발유,승용,소형,1991,32,32.75,3.32,10.60,0.01,0.01,...,3787.21,4976.74,0.00,0.00,380.37,31371.83,5360.43,3797.80,0.01,0.01
1149293,휘발유,승용,중형,1999,24,30.53,2.30,9.50,0.01,0.01,...,3395.43,3444.09,0.00,0.00,373.23,29251.97,3819.61,3404.93,0.01,0.01
1149294,휘발유,승용,중형,1995,28,30.03,2.26,9.34,0.01,0.01,...,3339.46,3387.31,0.00,0.00,372.21,28769.79,3761.78,3348.80,0.01,0.01


In [190]:
df2.loc[(df2['연료'] == '휘발유') & (df2['차종'] == '화물'), DF_col + check_E_HOT_col + check_E_cold_col + ['E_EVA_VOC'] + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,...,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_EVA_VOC,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1149310,휘발유,화물,경형,1993,30,46.49,9.66,26.25,0.01,0.01,...,0.00,0.00,0.00,0.00,394.49,46.49,404.16,26.25,0.01,0.01
1149313,휘발유,화물,경형,1995,28,54.20,11.27,30.60,0.01,0.01,...,0.00,0.00,0.00,0.00,408.27,54.20,419.54,30.60,0.01,0.01
1149321,휘발유,화물,경형,1993,30,40.12,8.34,22.65,0.01,0.01,...,0.00,0.00,0.00,0.00,383.09,40.12,391.44,22.65,0.01,0.01
1149322,휘발유,화물,소형,1993,30,41.91,3.17,13.16,0.01,0.01,...,0.00,0.00,0.00,0.00,397.05,41.91,400.22,13.16,0.01,0.01
1149454,휘발유,화물,소형,1996,27,56.39,4.27,17.70,0.01,0.01,...,0.00,0.00,0.00,0.00,426.65,56.39,430.92,17.70,0.01,0.01


## issue
- LPG : E_COLD_NOx 값 음수 -> 0으로 처리

In [191]:
df2.loc[df2['연료'] == 'LPG(액화석유가스)', DF_col + check_E_HOT_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_HOT_CO,E_HOT_HC,E_HOT_NOx,E_HOT_PM10,E_HOT_PM2_5,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1149292,LPG(액화석유가스),승합,중형,2000,23,49.61,1.57,14.49,0.01,0.01,0.00,0.00,0.00,0.00,0.00,49.61,1.57,14.49,0.01,0.01
1149359,LPG(액화석유가스),승용,중형,1996,27,51.98,2.54,13.94,0.01,0.00,11092.07,0.00,174.10,0.00,0.00,11144.04,176.64,13.94,0.01,0.00
1149391,LPG(액화석유가스),승용,중형,1998,25,67.74,3.62,13.72,0.01,0.01,14457.08,0.00,248.21,0.00,0.00,14524.82,251.83,13.72,0.01,0.01
1149399,LPG(액화석유가스),승용,중형,1994,29,45.77,6.37,10.82,0.01,0.01,9767.26,0.00,436.57,0.00,0.00,9813.03,442.94,10.82,0.01,0.01
1149429,LPG(액화석유가스),화물,소형,1998,25,79.57,4.25,16.11,0.01,0.01,0.00,0.00,0.00,0.00,0.00,79.57,4.25,16.11,0.01,0.01


In [192]:
df2.loc[df2['연료'] == 'LPG(액화석유가스)', DF_col + check_E_cold_col + check_E_col].head()

,연료,차종,차종유형,연식,차령,E_COLD_CO,E_COLD_NOx,E_COLD_HC,E_COLD_PM10,E_COLD_PM2_5,E_CO_total,E_HC_total,E_NOx_total,E_PM10_total,E_PM2_5_total
1149292,LPG(액화석유가스),승합,중형,2000,23,0.00,0.00,0.00,0.00,0.00,49.61,1.57,14.49,0.01,0.01
1149359,LPG(액화석유가스),승용,중형,1996,27,11092.07,0.00,174.10,0.00,0.00,11144.04,176.64,13.94,0.01,0.00
1149391,LPG(액화석유가스),승용,중형,1998,25,14457.08,0.00,248.21,0.00,0.00,14524.82,251.83,13.72,0.01,0.01
1149399,LPG(액화석유가스),승용,중형,1994,29,9767.26,0.00,436.57,0.00,0.00,9813.03,442.94,10.82,0.01,0.01
1149429,LPG(액화석유가스),화물,소형,1998,25,0.00,0.00,0.00,0.00,0.00,79.57,4.25,16.11,0.01,0.01


In [193]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '1일평균주행거리', 'VKT', 'DF_CO',
       'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx',
       'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO',
       'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d',
       'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c',
       'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total'],
      dtype='object')

In [194]:
df2['E_COLD_CO'].dtype

dtype('float64')

In [195]:
df2.loc[df2['E_CO_total'] < 0].shape

(0, 83)

In [196]:
df2.loc[df2['E_HC_total'] < 0].shape

(0, 83)

In [197]:
df2.loc[df2['E_PM10_total'] < 0].shape

(0, 83)

In [198]:
df2.loc[df2['E_PM2_5_total'] < 0].shape

(0, 83)

In [199]:
df2.loc[df2['E_NOx_total'] < 0, ['E_NOx_total', 'E_COLD_NOx']].shape

(0, 2)

## [출력] 배출량
- 연료 : 휘발유, 경유, LPG(액화석유가스)
- 물질 : CO, HC, NOx, PM10, PM2.5

In [200]:
df2.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '시군구명_수정', '1일평균주행거리', 'VKT', 'DF_CO',
       'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'E_HOT_CO', 'E_HOT_HC', 'E_HOT_NOx',
       'E_HOT_PM10', 'E_HOT_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'E_COLD_CO',
       'E_COLD_NOx', 'E_COLD_HC', 'E_COLD_PM10', 'E_COLD_PM2_5', 'e_d',
       'e_RHOT', 'e_SHOT', 'e_SWARM', 'e_fi', 'e_RWARM', 'R', 'S_fi', 'S_c',
       'E_EVA_VOC', 'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total',
       'E_PM2_5_total'],
      dtype='object')

In [201]:
df3 = df2[['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일', 
           '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식', 
           '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부', 
           '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2', 
           'DPF', '비고', 'Grade', 
           '차령', '1일평균주행거리', 'VKT', 'DF_CO', 'DF_HC', 'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC', 'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 
           'Beta(Estimated)', 'eCOLD/eHOT_CO', 'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 
           'e_d', 'e_RHOT', 'e_RWARM', 'R', 
           'E_HOT_CO', 
           'E_COLD_CO', 
           'E_CO_total', 
           'E_HOT_HC', 
           'E_COLD_HC', 
           'E_EVA_VOC', 
           'E_HC_total', 
           'E_HOT_NOx', 
           'E_COLD_NOx', 
           'E_NOx_total', 
           'E_HOT_PM10', 
           'E_COLD_PM10', 
           'E_PM10_total', 
           'E_HOT_PM2_5', 
           'E_COLD_PM2_5', 
           'E_PM2_5_total']]
df3.shape

(1497708, 77)

# dong_code 추가

## [병합] 4등급 + 3,5 등급

In [202]:
total = pd.concat([df3, g3_g5], ignore_index=True)
total.shape

(1502232, 77)

In [203]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '1일평균주행거리', 'VKT', 'DF_CO', 'DF_HC',
       'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'e_d', 'e_RHOT',
       'e_RWARM', 'R', 'E_HOT_CO', 'E_COLD_CO', 'E_CO_total', 'E_HOT_HC',
       'E_COLD_HC', 'E_EVA_VOC', 'E_HC_total', 'E_HOT_NOx', 'E_COLD_NOx',
       'E_NOx_total', 'E_HOT_PM10', 'E_COLD_PM10', 'E_PM10_total',
       'E_HOT_PM2_5', 'E_COLD_PM2_5', 'E_PM2_5_total'],
      dtype='object')

## 로드 법정동코드 표

In [204]:
name = 'tb_mapdata_gu(2023.04.20)'
file_name = f'{name}.xlsx'
code = pd.read_excel(os.path.join(raw_fold, file_name))

In [205]:
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dong_code    278 non-null    int64  
 1   ctprvn_code  278 non-null    int64  
 2   signgu_code  278 non-null    int64  
 3   sdstrd_code  278 non-null    int64  
 4   ctprvn_nm    278 non-null    object 
 5   signgu_nm    260 non-null    object 
 6   sdstrd_nm    0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 15.3+ KB


In [206]:
code = code.rename(columns={'ctprvn_nm':'시도명', 'signgu_nm':'시군구명'})
code.head()

,dong_code,ctprvn_code,signgu_code,sdstrd_code,시도명,시군구명,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


## 시군구명 수정
- 부천시 소사구 -> 부천시
- 부천시 오정구 -> 부천시
- 부천시 원미구 -> 부천시
- 세종특별자치시 -> nan
- 여주군 -> 여주시
- 청원군 -> 청주시
- 인천광역시 남구 -> 인천광역시 미추홀구

In [207]:
total['시군구명_수정'] = total['시군구명']
total['시군구명_수정'].isnull().sum()

0

In [208]:
total.loc[total['시군구명'] == '부천시 소사구', '시군구명'].shape

(6,)

In [209]:
total.loc[total['시군구명'] == '부천시 오정구', '시군구명'].shape

(19,)

In [210]:
total.loc[total['시군구명'] == '부천시 원미구', '시군구명'].shape

(20,)

In [211]:
total.loc[total['시군구명'] == '부천시 소사구', '시군구명_수정'] = '부천시'
total.loc[total['시군구명'] == '부천시 오정구', '시군구명_수정'] = '부천시'
total.loc[total['시군구명'] == '부천시 원미구', '시군구명_수정'] = '부천시'

In [212]:
total.loc[total['시군구명'] == '세종특별자치시', '시군구명'].shape

(8122,)

In [213]:
total.loc[total['시군구명'] == '세종특별자치시', '시군구명_수정'] = np.nan

In [214]:
total.loc[total['시군구명'] == '여주군', '시군구명'].shape

(1,)

In [215]:
total.loc[total['시군구명'] == '여주군', '시군구명_수정'] = '여주시'

In [216]:
total.loc[total['시군구명'] == '청원군', '시군구명'].shape

(4,)

In [217]:
total.loc[total['시군구명'] == '청원군', '시군구명_수정'] = '청주시'

In [218]:
total.loc[(total['시도명'] == '인천광역시') & (total['시군구명'] == '남구'), '시군구명'].shape

(9,)

In [219]:
total.loc[(total['시도명'] == '인천광역시') & (total['시군구명'] == '남구'), '시군구명_수정'] = '미추홀구'

In [220]:
set(total['시군구명_수정'].unique()) - set(code['시군구명'].unique())

set()

In [221]:
total['시군구명_수정'].isnull().sum()

8122

## 병합

In [222]:
total.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '1일평균주행거리', 'VKT', 'DF_CO', 'DF_HC',
       'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'e_d', 'e_RHOT',
       'e_RWARM', 'R', 'E_HOT_CO', 'E_COLD_CO', 'E_CO_total', 'E_HOT_HC',
       'E_COLD_HC', 'E_EVA_VOC', 'E_HC_total', 'E_HOT_NOx', 'E_COLD_NOx',
       'E_NOx_total', 'E_HOT_PM10', 'E_COLD_PM10', 'E_PM10_total',
       'E_HOT_PM2_5', 'E_COLD_PM2_5', 'E_PM2_5_total', '시군구명_수정'],
      dtype='object')

In [223]:
code.columns

Index(['dong_code', 'ctprvn_code', 'signgu_code', 'sdstrd_code', '시도명', '시군구명',
       'sdstrd_nm'],
      dtype='object')

In [224]:
total1 = total.merge(code[['시도명', '시군구명', 'dong_code']], left_on=['시도명', '시군구명_수정'], right_on=['시도명', '시군구명'], how='left')
total1.shape

(1502232, 80)

In [225]:
total1['dong_code'].isnull().sum()

0

In [226]:
total1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명_x', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '1일평균주행거리', 'VKT', 'DF_CO', 'DF_HC',
       'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'e_d', 'e_RHOT',
       'e_RWARM', 'R', 'E_HOT_CO', 'E_COLD_CO', 'E_CO_total', 'E_HOT_HC',
       'E_COLD_HC', 'E_EVA_VOC', 'E_HC_total', 'E_HOT_NOx', 'E_COLD_NOx',
       'E_NOx_total', 'E_HOT_PM10', 'E_COLD_PM10', 'E_PM10_total',
       'E_HOT_PM2_5', 'E_COLD_PM2_5', 'E_PM2_5_total', '시군구명_수정', '시군구명_y',
       'dong_code'],
      dtype='object')

In [227]:
total1 = total1.rename(columns={'시군구명_x':'시군구명'})
total1 = total1.drop(['시군구명_y'], axis=1)
total1.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '차령', '1일평균주행거리', 'VKT', 'DF_CO', 'DF_HC',
       'DF_PM', 'DF_NOx', 'R_CO', 'R_HC', 'R_PM', 'EFi_CO', 'EFi_HC',
       'EFi_NOx', 'EFi_PM10', 'EFi_PM2_5', 'Beta(Estimated)', 'eCOLD/eHOT_CO',
       'eCOLD/eHOT_NOx', 'eCOLD/eHOT_HC', 'eCOLD/eHOT_PM', 'e_d', 'e_RHOT',
       'e_RWARM', 'R', 'E_HOT_CO', 'E_COLD_CO', 'E_CO_total', 'E_HOT_HC',
       'E_COLD_HC', 'E_EVA_VOC', 'E_HC_total', 'E_HOT_NOx', 'E_COLD_NOx',
       'E_NOx_total', 'E_HOT_PM10', 'E_COLD_PM10', 'E_PM10_total',
       'E_HOT_PM2_5', 'E_COLD_PM2_5', 'E_PM2_5_total', '시군구명_수정', 'dong_code'],
      dtype='object')

In [228]:
final = total1[['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
                '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
                '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
                '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
                'DPF', '비고', 'Grade', '시군구명_수정', 'dong_code', 'E_CO_total', 'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total']]
final.shape

(1502232, 44)

## 출력(4등급)

In [229]:
final['등급_수정'].value_counts(dropna=False)

4.00    1497708
3.00       4397
5.00        127
Name: 등급_수정, dtype: int64

In [230]:
df4 =  final[final['등급_수정'] == 4].reset_index(drop=True)
df4.shape

(1497708, 44)

In [231]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.24_PM.05.44.35'

### csv

In [232]:
# about 32.8s
name = f'result(4등급)_CAPSS_배출량(Grade추가)(dong_code추가)({today_date})'
file_name = f'{name}.csv'
df4.to_csv(os.path.join(raw_fold, file_name), index=False)

In [233]:
name

'result(4등급)_CAPSS_배출량(Grade추가)(dong_code추가)(2023.04.24_PM.05.44.35)'

### excel

In [234]:
i = 1
f'{i:0>2}'

'01'

In [235]:
per = 500000
num = math.ceil(df4.shape[0] / per) + 1
num

4

In [249]:
# about 17m 7.1s
name = f'result(4등급)_CAPSS_배출량(Grade추가)(dong_code추가)({today_date})'
for i in range(1, num):
    df4.iloc[per*(i-1):per*i].to_excel(os.path.join(raw_fold, f'{name}{i:0>2}.xlsx'), index=False)

## 출력(전체)

In [237]:
final.shape

(1502232, 44)

### csv

In [238]:
# about 1m 27.1s
name = f'result(전체)_CAPSS_배출량(Grade추가)(dong_code추가)({today_date})'
file_name = f'{name}.csv'
final.to_csv(os.path.join(raw_fold, file_name), index=False)

In [239]:
name

'result(전체)_CAPSS_배출량(Grade추가)(dong_code추가)(2023.04.24_PM.05.44.35)'

### excel

In [240]:
per = 500000
num = math.ceil(final.shape[0] / per) + 1
num

5

In [250]:
# # about 30m 0.6s
name = f'result(전체)_CAPSS_배출량(Grade추가)(dong_code추가)({today_date})'
for i in range(1, num):
    final.iloc[per*(i-1):per*i].to_excel(os.path.join(raw_fold, f'{name}{i:0>2}.xlsx'), index=False)

# 시도/시군구별 배출량 합계

In [242]:
df4.columns

Index(['차대번호', '차량번호', '제원관리번호', '차종', '용도', '최초등록일', '연식', '제작일자', '검사유효일',
       '배출가스인증번호', '등급', '차명', '차종분류', '차종유형', '자동차형식', '제작사명', '연료', '엔진형식',
       '총중량', '적재중량', '엔진출력', '배기량', '법정동코드', '시도명', '시군구명', '소유자구분', '말소여부',
       '일치유무', '인증번호 수정 유무', '배인번호_수정', '등급_수정', 'DPF유무_수정', '배출가스', '등급2',
       'DPF', '비고', 'Grade', '시군구명_수정', 'dong_code', 'E_CO_total',
       'E_HC_total', 'E_NOx_total', 'E_PM10_total', 'E_PM2_5_total'],
      dtype='object')

In [244]:
groupby1 = df4.groupby(['연료', 'dong_code']).agg({'E_CO_total':'sum', 'E_HC_total':'sum', 'E_NOx_total':'sum', 'E_PM10_total':'sum', 'E_PM2_5_total':'sum'})
groupby1

E_CO_total  E_HC_total  E_NOx_total  E_PM10_total  \
연료          dong_code                                                       
LPG(액화석유가스) 1111000000   265388.41     8611.95       969.96          0.42   
            1114000000   363754.31    14436.73      1437.39          0.59   
            1117000000   353019.26    11903.67      1522.14          0.62   
            1120000000   471730.42     9750.80      1666.74          0.92   
            1121500000   821898.57    20558.00      1923.59          0.90   
...                            ...         ...          ...           ...   
휘발유         4887000000  5659824.64   996468.63    705979.40          1.82   
            4888000000  9462617.63  1710601.21   1212953.62          3.01   
            4889000000  6399008.05  1092054.19    770838.70          2.16   
            5011000000 75741581.70 12446572.94   8940125.62         24.66   
            5013000000 45726070.58  7872166.18   5468713.48         14.50   

                        E_PM2_5_total  
연료          dong_code                  
LPG(액화석유가스) 1111000000           0.39  
            1114000000           0.54  
            1117000000           0.57  
            1120000000           0.84  
            1121500000           0.83  
...                               ...  
휘발유         4887000000           1.68  
            4888000000           2.77  
            4889000000           1.99  
            5011000000          22.69  
            5013000000          13.34  

[751 rows x 5 columns]

## 출력

In [246]:
today_date = datetime.today().strftime("%Y.%m.%d_%p.%I.%M.%S")
today_date

'2023.04.24_PM.05.51.16'

In [248]:
groupby1.to_excel(os.path.join(analysis_fold, f'통계_4등급_result_시도_시군구별_CAPSS매연량합계({today_date}).xlsx'), sheet_name='시도_시군구')

# 코드 마지막